<p id='top'></p>

<h1 align='center'>PROJECT ON AVAILABLE PROPERTIES IN LONDON By BELLO ISMAHEEL</h1>

# PROJECT OUTLINE

1. [INTRODUCTION](#intro)
2. [DATA GATHERING](#collect)
    - [RIGHTMOVE](#right)
    - [ZOOPLA](#zoopla)
    - [ON THE MARKET](#otm)
3. [DATA WRANGLING](#wrangle)
    - [RIGHTMOVE](#wrightmove)
    - [ZOOPLA](#wzoopla)
    - [ON THE MARKET](#wotm)
4. [DATA MODELLING](#model)
5. [END NOTE](#report)


<p id='intro'</p>

## INTRODUCTION

This project is aimed at helping Mappa find her dream home with the Power of Data and Technology, the datasets used for this project was collected from [Rightmove](https://rightmove.co.uk) , [Zoopla](https://zoopla.co.uk) , and [On The Market](https://onthemove.co.uk) websites respectively. Data was collected on properties available for Sale and for rent. The features contained in each Dataset are:

- Transaction type (i.e. sale vs. rent - string)
- Bedrooms (integer)
- Bathrooms (integer)
- Description (free text string)
- Property type e.g. flat, detached house, terraced house
- Price e.g. 500,000 (typically integer)
- Location
- Agent (advertising the property)
- Listing source
- Listing URL


### Required packages

In [1]:
#!pip install geopy
#!pip install selenium
#pip install 'scrapfly-sdk'
import pandas as pd
import numpy as np
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from scrapfly import ScrapflyClient
from scrapfly import ScrapflyClient, ScrapeConfig
from urllib.parse import urljoin
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import warnings
warnings.filterwarnings('ignore')

<p id='collect'></p>

##  DATA GATHERING

<p id='right'></p>

### Webscraping rightmove.co.uk

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
service = Service('C:\\Users\\Simplicity\\chromedriver_win32')
driver = webdriver.Chrome(service=service, options=options)

#### For Sale

In [ ]:
# For Sale

# Set the URL for property search
url = 'https://www.rightmove.co.uk/property-for-sale.html'

# Navigate to the URL
driver.get(url)

# Pause execution for 10 seconds to allow the page to load
time.sleep(10)

# Find and clear the search box, then enter 'London' as the search query
search_box = driver.find_element(By.NAME, 'searchLocation')
search_box.clear()
search_box.send_keys('London')

# Pause execution for 5 seconds
time.sleep(5)

# Submit the search query by pressing the Enter key
search_box.send_keys(Keys.RETURN)

# Find the 'submit' button and click it
click = driver.find_element(By.ID, 'submit')
click.click()

# Pause execution for 10 seconds to allow the search results to load
time.sleep(10)

# Create an empty DataFrame to store property details
df = pd.DataFrame(columns=['Address', 'Type', 'Price', 'No_of_bedroom', 'No_of_bathroom', 'description',
                           'Agent_name', 'Agent_website', 'Agent_contact', 'Listing_url'])

# Loop to scrape property details from multiple pages
while True:
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Pause execution for 2 seconds
    time.sleep(2)
    
    # Get the page source and create a BeautifulSoup object
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    
    # Find all property elements on the page
    prop = re.findall(r"property-\d+", soup.prettify())
    
    # Extract details from each property element
    for i in prop:
        details = {}
        for j in soup.find('div', id=i, class_='l-searchResult is-list'):
            # Extract property details using various CSS selectors
            details['Address'] = [j.find('address', class_="propertyCard-address property-card-updates").text.strip()]
            details['Type'] = [j.find('span', class_="text").text]
            details['Price'] = [j.find('div', class_="propertyCard-priceValue").text]
            
            # Handle cases where the number of bedrooms or bathrooms is not available
            try:
                details['No_of_bedroom'] = [j.find('span', class_="no-svg-bed-icon bed-icon seperator").title.text]
            except AttributeError:
                details['No_of_bedroom'] = [np.NaN]
            try:
                details['No_of_bathroom'] = [j.find('span', class_="no-svg-bathroom-icon bathroom-icon seperator").title.text]
            except AttributeError:
                details['No_of_bathroom'] = [np.NaN]
            
            details['description'] = [j.find('span', itemprop="description").text]
            
            # Handle cases where agent details are not available
            try:
                details['Agent_name'] = [j.find('div', class_='propertyCard-branchLogo').a['title']]
            except AttributeError:
                details['Agent_name'] = [np.NaN]
            try:
                details['Agent_website'] = [re.search(r"([a-zA-Z]+[\W]+[a-zA-Z\.]+)", url).group() + j.find('div', class_='propertyCard-branchLogo').a['href']]
            except AttributeError:
                details['Agent_website'] = [np.NaN]
            try:
                details['Agent_contact'] = [re.search(r"([a-zA-Z]+[\W]+[a-zA-Z\.]+)", url).group() + j.find('a', class_='action mail-icon')['href']]
            except AttributeError:
                details['Agent_contact'] = [np.NaN]
            try:
                details['Listing_url'] = [re.search(r"([a-zA-Z]+[\W]+[a-zA-Z\.]+)", url).group() + j.find('a', class_='propertyCard-priceLink propertyCard-salePrice')['href']]
            except AttributeError:
                details['Listing_url'] = [np.NaN]
            
            # Create a DataFrame with the property details
            details = pd.DataFrame(details)
            df = pd.concat([df, details], ignore_index=True)
    
    # Find the next button for pagination
    button = driver.find_element(By.CSS_SELECTOR, 'button.pagination-button.pagination-direction.pagination-direction--next')
    
    # Check if the next button is disabled (indicating the last page)
    if driver.execute_script("return arguments[0].disabled;", button) == True:
        break
    else:
        # Click the next button to navigate to the next page
        driver.execute_script("arguments[0].click();", button)
    
    # Pause execution for 3 seconds
    time.sleep(3)

# Close the WebDriver
driver.quit()

# Display the first few rows of the DataFrame
df.head()


In [ ]:
df.to_csv('rightmove_sale.csv',index = False)
right_sales = df.copy()

#### For rent

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
service = Service('C:\\Users\\Simplicity\\chromedriver_win32')
driver = webdriver.Chrome(service=service, options=options)

In [ ]:
# For rent

# Set the URL for property search
url = 'https://www.rightmove.co.uk/property-to-rent.html'

# Navigate to the URL
driver.get(url)

# Find and clear the search box, then enter 'London' as the search query
search_box = driver.find_element(By.NAME, 'searchLocation')
search_box.clear()
search_box.send_keys('London')

# Pause execution for 5 seconds
time.sleep(5)

# Submit the search query by pressing the Enter key
search_box.send_keys(Keys.RETURN)

# Find the 'submit' button and click it
click = driver.find_element(By.ID, 'submit')
click.click()

# Create an empty DataFrame to store property details
df2 = pd.DataFrame(columns=['Address', 'Type', 'Price', 'Price_pw', 'No_of_bedroom', 'No_of_bathroom', 'description',
                            'Agent_name', 'Agent_website', 'Agent_contact', 'Listing_url'])

# Loop to scrape property details from multiple pages
while True:
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Pause execution for 2 seconds
    time.sleep(2)
    
    # Get the page source and create a BeautifulSoup object
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    
    # Find all property elements on the page
    prop = re.findall(r"property-\d+", soup.prettify())
    
    # Extract details from each property element
    for i in prop:
        details = {}
        for j in soup.find('div', id=i, class_='l-searchResult is-list'):
            # Extract property details using various CSS selectors
            details['Address'] = [j.find('address', class_="propertyCard-address property-card-updates").text.strip()]
            details['Type'] = [j.find('span', class_="text").text]
            details['Price'] = [j.find('span', class_="propertyCard-priceValue").text]
            details['Price_pw'] = [j.find('span', class_="propertyCard-secondaryPriceValue").text]
            
            # Handle cases where the number of bedrooms or bathrooms is not available
            try:
                details['No_of_bedroom'] = [j.find('span', class_="no-svg-bed-icon bed-icon seperator").title.text]
            except AttributeError:
                details['No_of_bedroom'] = [np.NaN]
            try:
                details['No_of_bathroom'] = [j.find('span', class_="no-svg-bathroom-icon bathroom-icon seperator").title.text]
            except AttributeError:
                details['No_of_bathroom'] = [np.NaN]
            
            details['description'] = [j.find('span', itemprop="description").text]
            
            # Handle cases where agent details are not available
            try:
                details['Agent_name'] = [j.find('div', class_='propertyCard-branchLogo').a['title']]
            except AttributeError:
                details['Agent_name'] = [np.NaN]
            try:
                details['Agent_website'] = [re.search(r"([a-zA-Z]+[\W]+[a-zA-Z\.]+)", url).group() + j.find('div', class_='propertyCard-branchLogo').a['href']]
            except AttributeError:
                details['Agent_website'] = [np.NaN]
            try:
                details['Agent_contact'] = [re.search(r"([a-zA-Z]+[\W]+[a-zA-Z\.]+)", url).group() + j.find('a', class_='action mail-icon')['href']]
            except AttributeError:
                details['Agent_contact'] = [np.NaN]
            
            # Extract the listing URL
            try:
                details['Listing_url'] = [re.search(r"([a-zA-Z]+[\W]+[a-zA-Z\.]+)", url).group() + j.find('a', class_='propertyCard-link property-card-updates')['href']]
            except AttributeError:
                details['Listing_url'] = [np.NaN]
            
            # Create a DataFrame with the property details
            details = pd.DataFrame(details)
            df2 = pd.concat([df2, details], ignore_index=True)
    
    # Find the next button for pagination
    button = driver.find_element(By.CSS_SELECTOR, 'button.pagination-button.pagination-direction.pagination-direction--next')
    
    # Check if the next button is disabled (indicating the last page)
    if driver.execute_script("return arguments[0].disabled;", button) == True:
        break
    else:
        # Click the next button to navigate to the next page
        driver.execute_script("arguments[0].click();", button)
    
    # Pause execution for 3 seconds
    time.sleep(3)

# Close the WebDriver
driver.quit()

# Display the first few rows of the DataFrame
df2.head()


In [ ]:
df2.to_csv('rightmove_rent.csv',index = False)
right_rent = df2.copy()

[GoToTop](#top)

<p id='zoopla'></p>

### Webscraping for zoopla.co.uk

### For sale.

In [ ]:
import time
import pandas as pd
import re
import numpy as np
from bs4 import BeautifulSoup
from scrapfly import ScrapflyClient, ScrapeConfig
from urllib.parse import urljoin

# Set up Scrapfly credentials
API_KEY = "MY API KEY"
client = ScrapflyClient(API_KEY)
count = 1
# Set the initial URL
url = 'https://www.zoopla.co.uk/for-sale/property/london/?q=London&search_source=home&pn={}'.format(count)
response = client.scrape(ScrapeConfig(url, country="GB", asp=True))
html = response.content
soup = BeautifulSoup(html, "lxml")
prop = re.findall(r"listing_\d+", soup.prettify())

# Create a DataFrame to store scraped data
sale_zoopla = pd.DataFrame(columns=['Address', 'Price', 'Description', 'Agent_name', 'Agent_website',
                                    'Agent_contact', 'Listing_url', 'No_of_Bedroom', 'No_of_Bathroom',
                                    'No_of_livingroom'])

# Loop to scrape property details from multiple pages
while True:
    # Extract details from each property element
    for i in prop:
        details = {}
        for j in soup.find('div', id=i, class_='f0xnzq2'):
            # Extract property details using CSS selectors
            Address = j.find('h3', class_="_1ankud52 _1ftx2fq9")
            if Address is not None:
                details['Address'] = Address.text.strip()

            # Extract price using CSS selector
            Price = j.find('div', class_="_170k6631")
            if Price is not None:
                details['Price'] = Price.text.strip()

            # Extract description using CSS selector
            description = j.find('h2', class_="_1ankud51 _1ftx2fq8")
            if description is not None:
                details['Description'] = description.text.strip()
            # Extract agent name and website using CSS selector
            Agent_name = j.find('div', class_='fas4ee0')
            if Agent_name is not None:
                img_tag = Agent_name.find('img', class_='_12bxhf70')
            if img_tag is not None:
                details['Agent_name'] = img_tag.get('alt')


            # Extract agent name and website using CSS selector
            Agent_website = j.find('div', class_='fas4ee0')
            if Agent_website is not None:
                details['Agent_website'] = urljoin(url, Agent_website.a['href'])

            # Extract agent contact using CSS selector
            Agent_contact = j.find('a', {'aria-label': 'Contact agent'})
            if Agent_contact is not None:
                details['Agent_contact'] = urljoin(url, Agent_contact['href'])

            # Extract listing URL using CSS selector
            Listing_url = j.find('a', class_='_1maljyt1')
            if Listing_url is not None:
                details['Listing_url'] = urljoin(url, Listing_url['href'])

            # Extract bedroom, bathroom, and living room information using CSS selector
            data = {'Bedrooms': np.NaN, 'Bathrooms': np.NaN, 'Living rooms': np.NaN}
            for item in j.find_all('li', class_='_3fbafd1'):
                value_element = re.search(r"\d+", item.text.strip()).group(0)
                label_element = item.find('span').text.strip()
                if label_element and value_element:
                    label = label_element
                    value = value_element
                    data[label] = value

            details['No_of_Bedroom'] = data['Bedrooms']
            details['No_of_Bathroom'] = data['Bathrooms']
            details['No_of_livingroom'] = data['Living rooms']

            # Create a DataFrame with the property details
            details = pd.DataFrame(details, index=[0])
            sale_zoopla = pd.concat([sale_zoopla, details], ignore_index=True)

    # Find the next page button element
    next_button = soup.find('a', class_="x8jo560 x8jo563 x8jo56a _1ftx2fq8", attrs={"aria-disabled": "false"})

    # Check if the next button is disabled (indicating the last page)
    if next_button is None:
        break
    else:
        # Get the URL for the next page
        next_url = next_button['href']
        # Construct the full URL using urljoin
        url = 'https://www.zoopla.co.uk/for-sale/property/london/?q=London&search_source=home&pn={}'.format(count)
        url = urljoin(url, next_url)
        # Make a request through Scrapfly to bypass anti-scraping measures
        response = client.scrape(ScrapeConfig(url, country="GB", asp=True))
        html = response.content

        soup = BeautifulSoup(html, "lxml")
        prop = re.findall(r"listing_\d+", soup.prettify())
        count+=1
        print(url)
        # Pause execution for a few seconds
        time.sleep(5)

# Display the scraped data
sale_zoopla


In [ ]:
zoopla_sale = sale_zoopla.copy()
sale_zoopla.to_csv('zoopla_sale.csv')

#### For rent

In [ ]:
# Set up Scrapfly credentials
API_KEY = "API KEY"
client = ScrapflyClient(API_KEY)
count = 1
# Set the initial URL
url = 'https://www.zoopla.co.uk/to-rent/property/london/?q=London&search_source=home&pn={}'.format(count)
response = client.scrape(ScrapeConfig(url, country="GB", asp=True))
html = response.content
soup = BeautifulSoup(html, "lxml")
prop = re.findall(r"listing_\d+", soup.prettify())

# Create a DataFrame to store scraped data
rent_zoopla = pd.DataFrame(columns=['Address', 'Price', 'Description', 'Agent_name', 'Agent_website',
                                    'Agent_contact', 'Listing_url', 'No_of_Bedroom', 'No_of_Bathroom',
                                    'No_of_livingroom'])

# Loop to scrape property details from multiple pages
while True:
    # Extract details from each property element
    for i in prop:
        details = {}
        for j in soup.find('div', id=i, class_='f0xnzq2'):
            # Extract property details using CSS selectors
            Address = j.find('h3', class_="_1ankud52 _1ftx2fq9")
            if Address is not None:
                details['Address'] = Address.text.strip()

            # Extract price using CSS selector
            Price = j.find('div', class_="_170k6631")
            if Price is not None:
                details['Price'] = Price.text.strip()

            # Extract description using CSS selector
            description = j.find('h2', class_="_1ankud51 _1ftx2fq8")
            if description is not None:
                details['Description'] = description.text.strip()
            # Extract agent name and website using CSS selector
            Agent_name = j.find('div', class_='fas4ee0')
            if Agent_name is not None:
                img_tag = Agent_name.find('img', class_='_12bxhf70')
            if img_tag is not None:
                details['Agent_name'] = img_tag.get('alt')


            # Extract agent name and website using CSS selector
            Agent_website = j.find('div', class_='fas4ee0')
            if Agent_website is not None:
                details['Agent_website'] = urljoin(url, Agent_website.a['href'])

            # Extract agent contact using CSS selector
            Agent_contact = j.find('a', {'aria-label': 'Contact agent'})
            if Agent_contact is not None:
                details['Agent_contact'] = urljoin(url, Agent_contact['href'])

            # Extract listing URL using CSS selector
            Listing_url = j.find('a', class_='_1maljyt1')
            if Listing_url is not None:
                details['Listing_url'] = urljoin(url, Listing_url['href'])

            # Extract bedroom, bathroom, and living room information using CSS selector
            data = {'Bedrooms': np.NaN, 'Bathrooms': np.NaN, 'Living rooms': np.NaN}
            for item in j.find_all('li', class_='_3fbafd1'):
                value_element = re.search(r"\d+", item.text.strip()).group(0)
                label_element = item.find('span').text.strip()
                if label_element and value_element:
                    label = label_element
                    value = value_element
                    data[label] = value

            details['No_of_Bedroom'] = data['Bedrooms']
            details['No_of_Bathroom'] = data['Bathrooms']
            details['No_of_livingroom'] = data['Living rooms']

            # Create a DataFrame with the property details
            details = pd.DataFrame(details, index=[0])
            rent_zoopla = pd.concat([rent_zoopla, details], ignore_index=True)

    # Find the next page button element
    next_button = soup.find('a', class_="x8jo560 x8jo563 x8jo56a _1ftx2fq8", attrs={"aria-disabled": "false"})

    # Check if the next button is disabled (indicating the last page)
    if next_button is None:
        break
    else:
        # Get the URL for the next page
        next_url = next_button['href']
        # Construct the full URL using urljoin
        url = 'https://www.zoopla.co.uk/to-rent/property/london/?q=London&search_source=home&pn={}'.format(count)
        url = urljoin(url, next_url)
        # Make a request through Scrapfly to bypass anti-scraping measures
        response = client.scrape(ScrapeConfig(url, country="GB", asp=True))
        html = response.content

        soup = BeautifulSoup(html, "lxml")
        prop = re.findall(r"listing_\d+", soup.prettify())
        count+=1
        print(url)
        # Pause execution for a few seconds
        time.sleep(5)

# Display the scraped data
rent_zoopla


In [ ]:
zoopla_rent = rent_zoopla.copy()
rent_zoopla.to_csv('zoopla_rent.csv')

[GoToTop](#top)

<p id='otm'></p>

## Webscraping onthemarket.co.uk

In [ ]:
options = webdriver.ChromeOptions()
#options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
service = Service('C:\\Users\\Simplicity\\chromedriver_win32')
driver = webdriver.Chrome(service=service, options=options)

### For sale

In [ ]:
url = 'https://www.onthemarket.com/for-sale/'
driver.get(url)
time.sleep(10)

In [ ]:
# Enter search query
search_box = driver.find_element(By.ID,'search-location-sale')
search_box.clear()
search_box.send_keys('London')
search_box.send_keys(Keys.RETURN)
time.sleep(5)

In [ ]:
# Create an empty DataFrame for OnTheMarket property data
otm_data = pd.DataFrame(columns=['Price', 'Address', 'description', 'No_of_bedroom', 'No_of_bathroom',
                                 'listing_url', 'agent_website', 'agent_name'])

# Loop to scrape property details from multiple pages
while True:
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Pause execution for 2 seconds
    time.sleep(2)
    
    # Get the page source and create a BeautifulSoup object
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    
    # Find all property elements on the page
    prop = re.findall(r"result-\d{3,}", soup.prettify())
    
    # Extract details from each property element
    for j in prop:
        otm = {}
        for i in soup.find('div', id=j):
            # Extract price using CSS selector
            price = i.find('div', class_='otm-Price')
            if price is not None:
                otm['Price'] = [price.find('a', class_='text-denim price mb-0 font-bold').text.strip()]
            
            # Extract address using CSS selector
            address = i.find('span', class_='address')
            if address is not None:
                otm['Address'] = [address.text.strip()]
            
            # Extract description using CSS selector
            description = i.find('span', class_='title')
            if description is not None:
                otm['description'] = [description.text.strip()]
            
            # Extract number of bedrooms and bathrooms using CSS selector
            bedroom_div = i.find('div', class_='otm-BedBathCount flex items-center')
            if bedroom_div is not None:
                otm['No_of_bedroom'] = [bedroom_div.find_all('div')[0].text]
                otm['No_of_bathroom'] = [bedroom_div.find_all('div')[1].text]
         
            # Extract listing URL using CSS selector
            listing_url = i.find('div', class_='agent-logo')
            if listing_url is not None:
                url = re.search(r"([a-zA-Z]+[\W]+[a-zA-Z\.]+)", url).group()
                otm['listing_url'] = [url + listing_url.find('a')['href']]
            
            # Extract agent website and name using CSS selector
            agent_website = i.find('div', class_='agent-logo')
            if agent_website is not None:
                agent_website = agent_website.find_next_sibling('div')
                otm['agent_website'] = [re.search(r"([a-zA-Z]+[\W]+[a-zA-Z\.]+)", url).group() + agent_website.find('small').a['href']]
                otm['agent_name'] = [agent_website.find('small').a.text]
                
        # Create a DataFrame with the property details
        otm = pd.DataFrame(otm)
        otm_data = pd.concat([otm_data, otm], ignore_index=True)
    
    try: 
        # Find the next button for pagination
        next_button = driver.find_element(By.CSS_SELECTOR, 'a[title="Next page"]')
        button_class = next_button.find_element(By.CSS_SELECTOR, 'button.otm-Button')
        
        # Check if the next button is disabled (indicating the last page)
        if driver.execute_script("return arguments[0].disabled;", button_class) == True:
            break
        else:
            # Scroll to the next button and click it
            driver.execute_script("arguments[0].scrollIntoView();", next_button)
            driver.execute_script("arguments[0].click();", button_class)
        
        # Pause execution for 10 seconds
        time.sleep(10)
    
    except NoSuchElementException:
        break

# Close the WebDriver
driver.quit()

# Display the first few rows of the OnTheMarket property data DataFrame
otm_data.head()


In [ ]:
otm_data

In [ ]:
otm_sale = otm_data.copy()
otm_data.to_csv('otm_sale.csv')

### For rent

In [ ]:
options = webdriver.ChromeOptions()
#options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
service = Service('C:\\Users\\Simplicity\\chromedriver_win32')
driver = webdriver.Chrome(service=service, options=options)

In [ ]:
url = 'https://www.onthemarket.com/to-rent/'
driver.get(url)
time.sleep(5)

In [ ]:
# Enter search query
search_box = driver.find_element(By.CLASS_NAME,'landing-search-input')
search_box.clear()
search_box.send_keys('London')
search_box.send_keys(Keys.RETURN)
time.sleep(5)

In [ ]:
# Create an empty DataFrame for storing OnTheMarket property data
otm_data = pd.DataFrame(columns=['Price', 'Address', 'description', 'No_of_bedroom', 'No_of_bathroom',
                                 'listing_url', 'agent_website', 'agent_name'])

# Loop to scrape property details from multiple pages
while True:
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Pause execution for 2 seconds
    time.sleep(2)
    
    # Get the page source and create a BeautifulSoup object
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    
    # Find all property elements on the page
    prop = re.findall(r"result-\d{3,}", soup.prettify())
    
    # Extract details from each property element
    for j in prop:
        otm = {}
        for i in soup.find('div', id=j):
            # Extract price using CSS selector
            price = i.find('div', class_='otm-Price')
            if price is not None:
                otm['Price'] = [price.find('a', class_='text-denim price mb-0 font-bold').text.strip()]
            
            # Extract address using CSS selector
            address = i.find('span', class_='address')
            if address is not None:
                otm['Address'] = [address.text.strip()]
            
            # Extract description using CSS selector
            description = i.find('span', class_='title')
            if description is not None:
                otm['description'] = [description.text.strip()]
            
            # Extract number of bedrooms and bathrooms using CSS selector
            bedroom_div = i.find('div', class_='otm-BedBathCount flex items-center')
            if bedroom_div is not None:
                otm['No_of_bedroom'] = [bedroom_div.find_all('div')[0].text]
                otm['No_of_bathroom'] = [bedroom_div.find_all('div')[1].text]
         
            # Extract listing URL using CSS selector
            listing_url = i.find('div', class_='agent-logo')
            if listing_url is not None:
                url = re.search(r"([a-zA-Z]+[\W]+[a-zA-Z\.]+)", url).group()
                otm['listing_url'] = [url + listing_url.find('a')['href']]
            
            # Extract agent website and name using CSS selector
            agent_website = i.find('div', class_='agent-logo')
            if agent_website is not None:
                agent_website = agent_website.find_next_sibling('div')
                otm['agent_website'] = [re.search(r"([a-zA-Z]+[\W]+[a-zA-Z\.]+)", url).group() + agent_website.find('small').a['href']]
                otm['agent_name'] = [agent_website.find('small').a.text]
                
        # Create a DataFrame with the property details
        otm = pd.DataFrame(otm)
        
        # Concatenate the property data DataFrame to the main DataFrame
        otm_data = pd.concat([otm_data, otm], ignore_index=True)
        
    try:
        # Find the next button for pagination
        next_button = driver.find_element(By.CSS_SELECTOR, 'a[title="Next page"]')
        button_class = next_button.find_element(By.CSS_SELECTOR, 'button.otm-Button')
        driver.execute_script("arguments[0].click();", button_class)
    
    except NoSuchElementException:
        # Break the loop if the next button is not found (indicating the last page)
        break

# Close the WebDriver
driver.quit()

# Display the first few rows of the OnTheMarket property data DataFrame
otm_data.head()


In [ ]:
otm_data

In [ ]:
otm_rent = otm_data.copy()
otm_data.to_csv('otm_rent.csv')

[GoToTop](#top)

<p id='wrangle'></p>

## DATA WRANGLING

All data collected will be imported for Data cleaning

## London district Postcode data

In [2]:
postcode_dist = pd.read_csv('postcode.csv')
post_list = postcode_dist['Postcode district'].tolist()
postcode_dist.head()

,Postcode district,Local Areas,Borough
0,BR1,"Bromley, Bickley, Downham",Bromley
1,BR2,"Bickley, Hayes, Bromley Common, Shortlands",Bromley
2,BR3,"Beckenham, Eden Park, Elmers End, Park Langley...",Bromley
3,BR4,West Wickham,Bromley
4,BR5,"Orpington, St Mary Cray, Petts Wood",Bromley


<p id='wrightmove'></p>

### RIGHTMOVE

In [3]:
#rightmove data
rightmove_sale = pd.read_csv('rightmove_sale.csv')
rightmove_rent = pd.read_csv('rightmove_rent.csv')

In [4]:
rightmove_sale.head()

,Address,Type,Price,No_of_bedroom,No_of_bathroom,description,Agent_name,Agent_website,Agent_contact,Listing_url
0,"Glyn Road, London, E5",Terraced,"£1,100,000",3 bedrooms,1 bathroom,A truly unique and stunning three bedroom peri...,"Stirling Ackroyd, Clapton",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1352692...
1,"One Hyde Park, Knightsbridge",Apartment,"£60,000,000",5 bedrooms,5 bathrooms,An exceptional exclusive five bedroom apartmen...,"The Cloister, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1301776...
2,"Lygon Place, Belgravia, SW1W",House,"£45,000,000",7 bedrooms,9 bathrooms,Ref. LOB0798 - Set behind a gated Belgravia dr...,"Beauchamp Estates Ltd, Mayfair - Resale",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1293029...
3,"Mayfair Freehold House, Park Lane Area, W1K",House,"£45,000,000",12 bedrooms,18 bathrooms,This stunning 12 bedroom (including 3 staff be...,"Luxury Living Homes International, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1303065...
4,"Pitt Street, Kensington, W8",House,"£44,000,000",6 bedrooms,5 bathrooms,Ref. LOB0625 - This striking and unique house ...,"Beauchamp Estates Ltd, Mayfair - Resale",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1318567...


In [5]:
rightmove_sale.shape

(1050, 10)

In [6]:
rightmove_sale.dtypes

Address           object
Type              object
Price             object
No_of_bedroom     object
No_of_bathroom    object
description       object
Agent_name        object
Agent_website     object
Agent_contact     object
Listing_url       object
dtype: object

In [7]:
rightmove_sale.Price = rightmove_sale.Price.str.extract('([\d{1,}]+)')
rightmove_sale.Price = rightmove_sale.Price.str.replace(',','')
rightmove_sale = rightmove_sale.dropna(subset = ['Price','No_of_bedroom','No_of_bathroom'],axis = 0)
rightmove_sale.Price = rightmove_sale.Price.astype('int')
rightmove_sale['Transaction_type'] = np.repeat('sales',rightmove_sale.shape[0])
rightmove_sale.rename(columns = {'Type':'Property_type'},inplace = True)
rightmove_sale['No_of_bedroom'] =  rightmove_sale['No_of_bedroom'].str.extract('(\d{1,})').astype('int')
rightmove_sale['No_of_bathroom'] =  rightmove_sale['No_of_bathroom'].str.extract('(\d{1,})').astype('int')

In [8]:
#Checking for duplicates
rightmove_sale.duplicated().sum()

0

In [9]:
#resetting index 
rightmove_sale.reset_index(drop = True,inplace = True)

In [10]:
rightmove_sale.head()

,Address,Property_type,Price,No_of_bedroom,No_of_bathroom,description,Agent_name,Agent_website,Agent_contact,Listing_url,Transaction_type
0,"Glyn Road, London, E5",Terraced,1100000,3,1,A truly unique and stunning three bedroom peri...,"Stirling Ackroyd, Clapton",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1352692...,sales
1,"One Hyde Park, Knightsbridge",Apartment,60000000,5,5,An exceptional exclusive five bedroom apartmen...,"The Cloister, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1301776...,sales
2,"Lygon Place, Belgravia, SW1W",House,45000000,7,9,Ref. LOB0798 - Set behind a gated Belgravia dr...,"Beauchamp Estates Ltd, Mayfair - Resale",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1293029...,sales
3,"Mayfair Freehold House, Park Lane Area, W1K",House,45000000,12,18,This stunning 12 bedroom (including 3 staff be...,"Luxury Living Homes International, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1303065...,sales
4,"Pitt Street, Kensington, W8",House,44000000,6,5,Ref. LOB0625 - This striking and unique house ...,"Beauchamp Estates Ltd, Mayfair - Resale",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1318567...,sales


In [11]:
# Extracting out the district Postcode present in the Address by using the list of District Postcode in london.
for i,j in enumerate(rightmove_sale['Address']):
    for k in post_list:
        if k in rightmove_sale['Address'][i]:
            rightmove_sale.loc[i,'Postcode_district'] = k
            break
        else:
            rightmove_sale.loc[i,'Postcode_district'] = np.NaN

In [12]:
rightmove_sale

,Address,Property_type,Price,No_of_bedroom,No_of_bathroom,description,Agent_name,Agent_website,Agent_contact,Listing_url,Transaction_type,Postcode_district
0,"Glyn Road, London, E5",Terraced,1100000,3,1,A truly unique and stunning three bedroom peri...,"Stirling Ackroyd, Clapton",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1352692...,sales,E5
1,"One Hyde Park, Knightsbridge",Apartment,60000000,5,5,An exceptional exclusive five bedroom apartmen...,"The Cloister, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1301776...,sales,NaN
2,"Lygon Place, Belgravia, SW1W",House,45000000,7,9,Ref. LOB0798 - Set behind a gated Belgravia dr...,"Beauchamp Estates Ltd, Mayfair - Resale",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1293029...,sales,SW1W
3,"Mayfair Freehold House, Park Lane Area, W1K",House,45000000,12,18,This stunning 12 bedroom (including 3 staff be...,"Luxury Living Homes International, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1303065...,sales,W1K
4,"Pitt Street, Kensington, W8",House,44000000,6,5,Ref. LOB0625 - This striking and unique house ...,"Beauchamp Estates Ltd, Mayfair - Resale",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1318567...,sales,W8
...,...,...,...,...,...,...,...,...,...,...,...,...
984,"William Mews, Knightsbridge, London, SW1X",Terraced,5950000,4,4,A fabulous low built four bedroom house,"Russell Simpson, Chelsea",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1327277...,sales,SW1X
985,"Davies Street, Mayfair, W1K",Flat,5950000,3,3,Ref. LOB0838 - A contemporary sixth-floor apar...,"Beauchamp Estates Ltd, Mayfair - Resale",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1326496...,sales,W1K
986,"Dunraven Street, Mayfair, W1K",Apartment,5950000,2,2,A beautiful and stylish penthouse apartment wh...,"Sandersons, Holland Park & Notting Hill",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/8486177...,sales,W1K
987,"Scarsdale Studios, 21a Stratford Road, Kensing...",House,5950000,4,3,Unique four bedroom house in a former artist’s...,"Russell Simpson, Kensington",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1341741...,sales,W8


In [13]:
#Filling null values with None
rightmove_sale['Postcode_district'].fillna('None',inplace = True)

In [14]:
#Making a copy of the District code yet to be gotten.
rightmove_sale_copy = rightmove_sale.query('Postcode_district == "None"')

In [15]:
#Dropping the index where we have district code to be null
rightmove_sale.drop(rightmove_sale_copy.index,inplace = True)

In [16]:
#Resetting index of the copied data
rightmove_sale_copy.reset_index(drop = True,inplace = True)

In [17]:
rightmove_sale.query('Postcode_district == "None"')

,Address,Property_type,Price,No_of_bedroom,No_of_bathroom,description,Agent_name,Agent_website,Agent_contact,Listing_url,Transaction_type,Postcode_district


In [18]:
#Using geolocator to get other district postcodes
geolocator = Nominatim(user_agent="postcode_extractor")
for i, j in enumerate(rightmove_sale_copy['Address']):
    location = geolocator.geocode(rightmove_sale_copy['Address'][i], country_codes="GB", addressdetails=True, timeout=10)
    if location is not None:
        # Extract the postcode from the location address details
        postcode = location.raw.get('address', {}).get('postcode')
        if postcode:
            # Extract the district postcode from the full postcode
            rightmove_sale_copy.loc[i, 'Postcode_district'] = postcode.split()[0]
        else:
            # If postcode is not available, set district postcode as 'None'
            rightmove_sale_copy.loc[i, 'Postcode_district'] = 'None'
    else:
        # If location is not found, set district postcode as 'None'
        rightmove_sale_copy.loc[i, 'Postcode_district'] = 'None'


In [19]:
rightmove_sale = rightmove_sale.query('Postcode_district != "None"')

In [20]:
#Checking district code which are yet to be gotten
righmove_sale_copy = rightmove_sale_copy.query('Postcode_district != "None"')

In [21]:
rightmove_sale_copy.query('Postcode_district != "None"')

,Address,Property_type,Price,No_of_bedroom,No_of_bathroom,description,Agent_name,Agent_website,Agent_contact,Listing_url,Transaction_type,Postcode_district
0,"One Hyde Park, Knightsbridge",Apartment,60000000,5,5,An exceptional exclusive five bedroom apartmen...,"The Cloister, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1301776...,sales,SW1X
1,"Royal Westminster, London",Detached,35000000,7,5,A very rare find this luxurious circa 11000 SQ...,"Luxury Living Homes International, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1344499...,sales,SW1Y
2,"South Street, London",Terraced,35000000,5,6,"An elegant stone-dressed, red-brick, five-stor...","Wetherell, Mayfair",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1281503...,sales,TN16
3,"One Hyde Park, Knightsbridge",Apartment,30000000,3,3,The Cloister are pleased to present this stunn...,"The Cloister, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1343810...,sales,SW1X
5,Culross House,House,26950000,5,6,"A newly constructed Mayfair Mansion, designed ...","London House, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1326125...,sales,W10
...,...,...,...,...,...,...,...,...,...,...,...,...
183,"Kew Bridge Road, Brentford",Apartment,435000,1,1,Luxury apartment - WITH OWN SECURE GATED ALLOC...,"Barnard Marcus, Chiswick",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1343518...,sales,W4
184,"Trevor Place, London",Terraced,5999950,4,3,Trevor Place is a stunning freehold property i...,"Chatterton Rees, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1315975...,sales,SW7
186,"Carnation Way,London",Apartment,5960000,4,4,Luxury Living Homes is pleased to present an i...,"Luxury Living Homes International, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1334994...,sales,SW8
187,"Uphill Road, Mill Hill",Detached,5950000,6,5,A beautifully designed Tudor style home in the...,"Real Estates, Totteridge",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1340668...,sales,NW7


In [22]:
rightmove_sale.dtypes

Address              object
Property_type        object
Price                 int32
No_of_bedroom         int32
No_of_bathroom        int32
description          object
Agent_name           object
Agent_website        object
Agent_contact        object
Listing_url          object
Transaction_type     object
Postcode_district    object
dtype: object

In [23]:
#Getting the final data.
rightmove_sale = pd.concat([rightmove_sale,rightmove_sale_copy],ignore_index = True)

In [24]:
rightmove_sale.rename(columns = {'No_of_bedroom':'No_of_Bedroom','No_of_bathroom':'No_of_Bathroom',
                                'description':'Description'},inplace = True)

In [25]:
rightmove_sale

,Address,Property_type,Price,No_of_Bedroom,No_of_Bathroom,Description,Agent_name,Agent_website,Agent_contact,Listing_url,Transaction_type,Postcode_district
0,"Glyn Road, London, E5",Terraced,1100000,3,1,A truly unique and stunning three bedroom peri...,"Stirling Ackroyd, Clapton",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1352692...,sales,E5
1,"Lygon Place, Belgravia, SW1W",House,45000000,7,9,Ref. LOB0798 - Set behind a gated Belgravia dr...,"Beauchamp Estates Ltd, Mayfair - Resale",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1293029...,sales,SW1W
2,"Mayfair Freehold House, Park Lane Area, W1K",House,45000000,12,18,This stunning 12 bedroom (including 3 staff be...,"Luxury Living Homes International, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1303065...,sales,W1K
3,"Pitt Street, Kensington, W8",House,44000000,6,5,Ref. LOB0625 - This striking and unique house ...,"Beauchamp Estates Ltd, Mayfair - Resale",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1318567...,sales,W8
4,"Pitt Street, London, W8",Detached,44000000,7,5,"A rare opportunity to own this unique, contemp...","Savills, Kensington",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1318464...,sales,W8
...,...,...,...,...,...,...,...,...,...,...,...,...
984,"Trevor Place, London",Terraced,5999950,4,3,Trevor Place is a stunning freehold property i...,"Chatterton Rees, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1315975...,sales,SW7
985,"Elm Walk, Hampstead",Detached,5995999,6,4,An imposing detached double fronted house offe...,"Goldschmidt & Howland, Hampstead - Sales",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1281999...,sales,None
986,"Carnation Way,London",Apartment,5960000,4,4,Luxury Living Homes is pleased to present an i...,"Luxury Living Homes International, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1334994...,sales,SW8
987,"Uphill Road, Mill Hill",Detached,5950000,6,5,A beautifully designed Tudor style home in the...,"Real Estates, Totteridge",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1340668...,sales,NW7


In [26]:
rightmove_sale.to_csv('rightmove_sale_master.csv',index = False)

In [27]:
rightmove_rent.head()

,Address,Type,Price,Price_pw,No_of_bedroom,No_of_bathroom,description,Agent_name,Agent_website,Agent_contact,Listing_url
0,"Blythwood Road, Stroud Green, London, N4",End of Terrace,"£5,250 pcm","£1,212 pw",5 bedrooms,2 bathrooms,NO DEPOSIT OPTION AVAILABLE. Lovely five bedro...,"Martyn Gerrard, Crouch End",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1352645...
1,"Kingfisher Heights, London, E16",Flat,"£1,850 pcm",£427 pw,1 bedroom,1 bathroom,Property Description: This beautiful one bedro...,"OpenRent, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1354085...
2,"Torriano Avenue, London, NW5",Flat,"£4,500 pcm","£1,038 pw",1 bedroom,1 bathroom,A wonderfully stylish 1 bedroom apartment loca...,"OpenRent, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1354085...
3,"Ringstead Road, London, SE6",Flat,"£1,460 pcm",£337 pw,1 bedroom,1 bathroom,We are proud to offer this delightful 1 bedroo...,"OpenRent, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1354085...
4,"A Millway, London, NW7",Maisonette,"£1,800 pcm",£415 pw,2 bedrooms,1 bathroom,We are proud to offer this delightful 2 bedroo...,"OpenRent, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1354085...


In [28]:
rightmove_rent = rightmove_rent.dropna(subset = ['Price','Price_pw','No_of_bedroom','No_of_bathroom'],axis = 0)
cols = ['Price','Price_pw']
for i in cols:
    rightmove_rent[i] = rightmove_rent[i].str.extract('([\d{1,}]+)')
    rightmove_rent[i] = rightmove_rent[i].str.replace(',','')
    rightmove_rent = rightmove_rent.dropna(subset = [i],axis = 0)
    rightmove_rent[i] = rightmove_rent[i].astype(int)
rightmove_rent.rename(columns = {'Price':'Price_pcm'},inplace = True)
rightmove_rent['No_of_bedroom'] =  rightmove_rent['No_of_bedroom'].str.extract('(\d{1,})').astype('int')
rightmove_rent['No_of_bathroom'] =  rightmove_rent['No_of_bathroom'].str.extract('(\d{1,})').astype('int')

In [29]:
cols = ['Price','Price_pw']

In [30]:
rightmove_rent.dtypes

Address           object
Type              object
Price_pcm          int32
Price_pw           int32
No_of_bedroom      int32
No_of_bathroom     int32
description       object
Agent_name        object
Agent_website     object
Agent_contact     object
Listing_url       object
dtype: object

In [31]:
rightmove_rent.head()

,Address,Type,Price_pcm,Price_pw,No_of_bedroom,No_of_bathroom,description,Agent_name,Agent_website,Agent_contact,Listing_url
0,"Blythwood Road, Stroud Green, London, N4",End of Terrace,5250,1212,5,2,NO DEPOSIT OPTION AVAILABLE. Lovely five bedro...,"Martyn Gerrard, Crouch End",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1352645...
1,"Kingfisher Heights, London, E16",Flat,1850,427,1,1,Property Description: This beautiful one bedro...,"OpenRent, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1354085...
2,"Torriano Avenue, London, NW5",Flat,4500,1038,1,1,A wonderfully stylish 1 bedroom apartment loca...,"OpenRent, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1354085...
3,"Ringstead Road, London, SE6",Flat,1460,337,1,1,We are proud to offer this delightful 1 bedroo...,"OpenRent, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1354085...
4,"A Millway, London, NW7",Maisonette,1800,415,2,1,We are proud to offer this delightful 2 bedroo...,"OpenRent, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1354085...


In [32]:
#Checking for duplicates
rightmove_rent.duplicated().sum()

0

In [33]:
#resetting index
rightmove_rent.reset_index(drop = True,inplace = True)

In [34]:
# Extracting out the district Postcode present in the Address by using the list of District Postcode in London.
for i, j in enumerate(rightmove_rent['Address']):
    for k in post_list:
        if k in rightmove_rent['Address'][i]:
            rightmove_rent.loc[i, 'Postcode_district'] = k
            break
        else:
            rightmove_rent.loc[i, 'Postcode_district'] = np.NaN

# Filling null values with "None"
rightmove_rent['Postcode_district'].fillna('None', inplace=True)

# Making a copy of the District code yet to be gotten.
rightmove_rent_copy = rightmove_rent.query('Postcode_district == "None"')

# Dropping the index where we have district code to be null
rightmove_rent.drop(rightmove_rent_copy.index, inplace=True)

# Resetting index of the copied data
rightmove_rent_copy.reset_index(drop=True, inplace=True)

In [35]:

# Using geolocator to get other District codes
geolocator = Nominatim(user_agent="postcode_extractor")
for i, j in enumerate(rightmove_rent_copy['Address']):
    location = geolocator.geocode(rightmove_rent_copy['Address'][i], country_codes="GB", addressdetails=True,timeout = 10)
    if location is not None:
        postcode = location.raw.get('address', {}).get('postcode')
        if postcode:
            rightmove_rent_copy.loc[i, 'Postcode_district'] = postcode.split()[0]
        else:
            rightmove_rent_copy.loc[i, 'Postcode_district'] = 'None'
    else:
        rightmove_rent_copy.loc[i, 'Postcode_district'] = 'None'

# Checking district codes which are yet to be gotten
rightmove_rent_copy = rightmove_rent_copy.query('Postcode_district != "None"')

# Getting the final data
rightmove_rent = pd.concat([rightmove_rent, rightmove_rent_copy], ignore_index=True)

rightmove_rent


,Address,Type,Price_pcm,Price_pw,No_of_bedroom,No_of_bathroom,description,Agent_name,Agent_website,Agent_contact,Listing_url,Postcode_district
0,"Blythwood Road, Stroud Green, London, N4",End of Terrace,5250,1212,5,2,NO DEPOSIT OPTION AVAILABLE. Lovely five bedro...,"Martyn Gerrard, Crouch End",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1352645...,N4
1,"Kingfisher Heights, London, E16",Flat,1850,427,1,1,Property Description: This beautiful one bedro...,"OpenRent, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1354085...,E1
2,"Torriano Avenue, London, NW5",Flat,4500,1038,1,1,A wonderfully stylish 1 bedroom apartment loca...,"OpenRent, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1354085...,NW5
3,"Ringstead Road, London, SE6",Flat,1460,337,1,1,We are proud to offer this delightful 1 bedroo...,"OpenRent, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1354085...,E6
4,"A Millway, London, NW7",Maisonette,1800,415,2,1,We are proud to offer this delightful 2 bedroo...,"OpenRent, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1354085...,NW7
...,...,...,...,...,...,...,...,...,...,...,...,...
835,"94 White Horse Road, Limehouse",Apartment,2561,591,2,2,ea2 are pleased to offer to let this 2 bedroom...,"ea2 Estate Agency, Wapping",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1353425...,E1
836,Hightail Point,Apartment,2695,622,2,2,"In East Village’s newest, most striking develo...","Get Living, East Village by Get Living",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1353425...,E20
837,"Albyn Road, Deptford",Terraced,3250,750,4,1,A four double bedroom Victorian house includin...,"Dexters, New Cross",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1353423...,SE8
838,"Hazelwood Lane, Palmers Green",Maisonette,2197,507,3,1,Available from the 26th July on an unfurnished...,"Outlet Residential, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1353422...,N13


In [36]:
# Checking district codes which are yet to be gotten
rightmove_rent_copy = rightmove_rent_copy.query('Postcode_district != "None"')

# Getting the final data
rightmove_rent = pd.concat([rightmove_rent, rightmove_rent_copy], ignore_index=True)

rightmove_rent

,Address,Type,Price_pcm,Price_pw,No_of_bedroom,No_of_bathroom,description,Agent_name,Agent_website,Agent_contact,Listing_url,Postcode_district
0,"Blythwood Road, Stroud Green, London, N4",End of Terrace,5250,1212,5,2,NO DEPOSIT OPTION AVAILABLE. Lovely five bedro...,"Martyn Gerrard, Crouch End",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1352645...,N4
1,"Kingfisher Heights, London, E16",Flat,1850,427,1,1,Property Description: This beautiful one bedro...,"OpenRent, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1354085...,E1
2,"Torriano Avenue, London, NW5",Flat,4500,1038,1,1,A wonderfully stylish 1 bedroom apartment loca...,"OpenRent, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1354085...,NW5
3,"Ringstead Road, London, SE6",Flat,1460,337,1,1,We are proud to offer this delightful 1 bedroo...,"OpenRent, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1354085...,E6
4,"A Millway, London, NW7",Maisonette,1800,415,2,1,We are proud to offer this delightful 2 bedroo...,"OpenRent, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1354085...,NW7
...,...,...,...,...,...,...,...,...,...,...,...,...
945,"94 White Horse Road, Limehouse",Apartment,2561,591,2,2,ea2 are pleased to offer to let this 2 bedroom...,"ea2 Estate Agency, Wapping",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1353425...,E1
946,Hightail Point,Apartment,2695,622,2,2,"In East Village’s newest, most striking develo...","Get Living, East Village by Get Living",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1353425...,E20
947,"Albyn Road, Deptford",Terraced,3250,750,4,1,A four double bedroom Victorian house includin...,"Dexters, New Cross",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1353423...,SE8
948,"Hazelwood Lane, Palmers Green",Maisonette,2197,507,3,1,Available from the 26th July on an unfurnished...,"Outlet Residential, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1353422...,N13


In [37]:
rightmove_rent.rename(columns = {'Type':'Property_type'},inplace = True)
rightmove_rent['Transaction_type'] = np.repeat('rent',rightmove_rent.shape[0])

In [38]:
rightmove_rent.rename(columns = {'No_of_bedroom':'No_of_Bedroom','No_of_bathroom':'No_of_Bathroom',
                                'description':'Description'},inplace = True)

In [39]:
rightmove_rent.to_csv('rightmove_rent_master.csv',index = False)

<p id='wzoopla'></p>

### ZOOPLA

In [40]:
#zoopla data
zoopla_sale = pd.read_csv('zoopla_sale.csv')
zoopla_rent = pd.read_csv('zoopla_rent.csv')

In [41]:
zoopla_sale.drop(columns = ['Unnamed: 0'],inplace = True)

In [42]:
zoopla_sale.head()

,Address,Price,Description,Agent_name,Agent_website,Agent_contact,Listing_url,No_of_Bedroom,No_of_Bathroom,No_of_livingroom
0,"Arterberry Road, London SW20","Guide price£450,000",1 bed flat for sale,Lucie White & Company,https://www.zoopla.co.uk/find-agents/branch/lu...,https://www.zoopla.co.uk/for-sale/details/cont...,https://www.zoopla.co.uk/for-sale/details/6467...,1.0,1.0,1.0
1,"Arterberry Road, London SW20","Guide price£450,000",1 bed flat for sale,Lucie White & Company,https://www.zoopla.co.uk/find-agents/branch/lu...,https://www.zoopla.co.uk/for-sale/details/cont...,https://www.zoopla.co.uk/for-sale/details/6467...,NaN,NaN,NaN
2,"""Block C1 CD13 So - Plot 260"" at Oliver Road, ...","From£665,000",3 bed flat for sale,Taylor Wimpey - Coronation Square,https://www.zoopla.co.uk/new-homes/developers/...,https://www.zoopla.co.uk/new-homes/details/con...,https://www.zoopla.co.uk/new-homes/details/647...,3.0,NaN,NaN
3,"Colegrave Road, Stratford, London E15","£295,000",1 bed flat for sale,Portico Property - Leyton,https://www.zoopla.co.uk/find-agents/branch/po...,https://www.zoopla.co.uk/for-sale/details/cont...,https://www.zoopla.co.uk/for-sale/details/6473...,1.0,1.0,1.0
4,"Colegrave Road, Stratford, London E15","£295,000",1 bed flat for sale,Portico Property - Leyton,https://www.zoopla.co.uk/find-agents/branch/po...,https://www.zoopla.co.uk/for-sale/details/cont...,https://www.zoopla.co.uk/for-sale/details/6473...,NaN,NaN,NaN


In [43]:
zoopla_sale.dtypes

Address              object
Price                object
Description          object
Agent_name           object
Agent_website        object
Agent_contact        object
Listing_url          object
No_of_Bedroom       float64
No_of_Bathroom      float64
No_of_livingroom    float64
dtype: object

In [44]:
zoopla_sale = zoopla_sale.dropna(subset = ['Price','No_of_Bedroom','No_of_Bathroom'],axis = 0)
zoopla_sale['Price'] = zoopla_sale['Price'].str.extract('([\d{1,}]+)')
zoopla_sale['Price'] = zoopla_sale['Price'].str.replace(',','')
zoopla_sale['Price'] = zoopla_sale['Price'].astype('int')
zoopla_sale['No_of_Bedroom'] = zoopla_sale['No_of_Bedroom'].astype('int')
zoopla_sale['No_of_Bathroom'] = zoopla_sale['No_of_Bathroom'].astype('int') 

In [45]:
zoopla_sale.dtypes

Address              object
Price                 int32
Description          object
Agent_name           object
Agent_website        object
Agent_contact        object
Listing_url          object
No_of_Bedroom         int32
No_of_Bathroom        int32
No_of_livingroom    float64
dtype: object

In [46]:
zoopla_sale['Property_type'] = zoopla_sale.Description.str.extract('([a-zA-Z]+\s+[a-zA-Z.\-]+)',expand = True)

In [47]:
zoopla_sale['Property_type'] = zoopla_sale['Property_type'].str.replace('bed','')

In [48]:
zoopla_sale['Transaction_type'] = np.repeat('sale',zoopla_sale.shape[0]) 

In [49]:
#checking for duplicates
zoopla_sale.duplicated().sum()

22

In [50]:
#Resetting index
zoopla_sale.reset_index(drop = True,inplace = True)
zoopla_sale.head()

,Address,Price,Description,Agent_name,Agent_website,Agent_contact,Listing_url,No_of_Bedroom,No_of_Bathroom,No_of_livingroom,Property_type,Transaction_type
0,"Arterberry Road, London SW20",450000,1 bed flat for sale,Lucie White & Company,https://www.zoopla.co.uk/find-agents/branch/lu...,https://www.zoopla.co.uk/for-sale/details/cont...,https://www.zoopla.co.uk/for-sale/details/6467...,1,1,1.0,flat,sale
1,"Colegrave Road, Stratford, London E15",295000,1 bed flat for sale,Portico Property - Leyton,https://www.zoopla.co.uk/find-agents/branch/po...,https://www.zoopla.co.uk/for-sale/details/cont...,https://www.zoopla.co.uk/for-sale/details/6473...,1,1,1.0,flat,sale
2,"Chilham Close, Perivale, Greenford UB6",799950,4 bed semi-detached house for sale,Peter Gamble & Co,https://www.zoopla.co.uk/find-agents/branch/pe...,https://www.zoopla.co.uk/for-sale/details/cont...,https://www.zoopla.co.uk/for-sale/details/6473...,4,2,2.0,semi-detached,sale
3,"Frensham Close, Southall UB1",289950,2 bed flat for sale,Match A Property,https://www.zoopla.co.uk/find-agents/branch/ma...,https://www.zoopla.co.uk/for-sale/details/cont...,https://www.zoopla.co.uk/for-sale/details/6473...,2,1,1.0,flat,sale
4,"Berens Road, Orpington BR5",400000,3 bed semi-detached house for sale,Proctors - Petts Wood,https://www.zoopla.co.uk/find-agents/branch/pr...,https://www.zoopla.co.uk/for-sale/details/cont...,https://www.zoopla.co.uk/for-sale/details/6473...,3,1,1.0,semi-detached,sale


In [51]:
# Extracting out the district Postcode present in the Address by using the list of District Postcode in London.
for i, j in enumerate(zoopla_sale['Address']):
    for k in post_list:
        if k in zoopla_sale['Address'][i]:
            zoopla_sale.loc[i, 'Postcode_district'] = k
            break
        else:
            zoopla_sale.loc[i, 'Postcode_district'] = np.NaN

# Filling null values with "None"
zoopla_sale['Postcode_district'].fillna('None', inplace=True)

# Making a copy of the District code yet to be gotten.
zoopla_sale_copy = zoopla_sale.query('Postcode_district == "None"')

# Dropping the index where we have district code to be null
zoopla_sale.drop(zoopla_sale_copy.index, inplace=True)

# Resetting index of the copied data
zoopla_sale_copy.reset_index(drop=True, inplace=True)

# Using geolocator to get other District codes
geolocator = Nominatim(user_agent="postcode_extractor")
for i, j in enumerate(zoopla_sale_copy['Address']):
    location = geolocator.geocode(zoopla_sale_copy['Address'][i], country_codes="GB", addressdetails=True,timeout=10)
    if location is not None:
        postcode = location.raw.get('address', {}).get('postcode')
        if postcode:
            zoopla_sale_copy.loc[i, 'Postcode_district'] = postcode.split()[0]
        else:
            zoopla_sale_copy.loc[i, 'Postcode_district'] = 'None'
    else:
        zoopla_sale_copy.loc[i, 'Postcode_district'] = 'None'

# Checking district codes which are yet to be gotten
zoopla_sale_copy = zoopla_sale_copy.query('Postcode_district != "None"')

# Getting the final data
zoopla_sale = pd.concat([zoopla_sale, zoopla_sale_copy], ignore_index=True)

zoopla_sale


,Address,Price,Description,Agent_name,Agent_website,Agent_contact,Listing_url,No_of_Bedroom,No_of_Bathroom,No_of_livingroom,Property_type,Transaction_type,Postcode_district
0,"Arterberry Road, London SW20",450000,1 bed flat for sale,Lucie White & Company,https://www.zoopla.co.uk/find-agents/branch/lu...,https://www.zoopla.co.uk/for-sale/details/cont...,https://www.zoopla.co.uk/for-sale/details/6467...,1,1,1.0,flat,sale,SW2
1,"Colegrave Road, Stratford, London E15",295000,1 bed flat for sale,Portico Property - Leyton,https://www.zoopla.co.uk/find-agents/branch/po...,https://www.zoopla.co.uk/for-sale/details/cont...,https://www.zoopla.co.uk/for-sale/details/6473...,1,1,1.0,flat,sale,E1
2,"Chilham Close, Perivale, Greenford UB6",799950,4 bed semi-detached house for sale,Peter Gamble & Co,https://www.zoopla.co.uk/find-agents/branch/pe...,https://www.zoopla.co.uk/for-sale/details/cont...,https://www.zoopla.co.uk/for-sale/details/6473...,4,2,2.0,semi-detached,sale,UB6
3,"Frensham Close, Southall UB1",289950,2 bed flat for sale,Match A Property,https://www.zoopla.co.uk/find-agents/branch/ma...,https://www.zoopla.co.uk/for-sale/details/cont...,https://www.zoopla.co.uk/for-sale/details/6473...,2,1,1.0,flat,sale,UB1
4,"Berens Road, Orpington BR5",400000,3 bed semi-detached house for sale,Proctors - Petts Wood,https://www.zoopla.co.uk/find-agents/branch/pr...,https://www.zoopla.co.uk/for-sale/details/cont...,https://www.zoopla.co.uk/for-sale/details/6473...,3,1,1.0,semi-detached,sale,BR5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
617,"Wood Close, London NW9",650000,4 bed terraced house for sale,Blue Ocean Property Consultants,https://www.zoopla.co.uk/find-agents/branch/bl...,https://www.zoopla.co.uk/for-sale/details/cont...,https://www.zoopla.co.uk/for-sale/details/6472...,4,2,1.0,terraced,sale,NW9
618,"Thames Avenue, London UB6",1000000,4 bed terraced house for sale,"Fine & Country - Harrow, Northwood & Pinner",https://www.zoopla.co.uk/find-agents/branch/fi...,https://www.zoopla.co.uk/for-sale/details/cont...,https://www.zoopla.co.uk/for-sale/details/6472...,4,3,2.0,terraced,sale,UB6
619,"Verwood Road, Harrow HA2",675000,4 bed semi-detached house for sale,Major Estates Sales & Lettings,https://www.zoopla.co.uk/find-agents/branch/ma...,https://www.zoopla.co.uk/for-sale/details/cont...,https://www.zoopla.co.uk/for-sale/details/6472...,4,1,1.0,semi-detached,sale,HA2
620,"Brigadier Hill, Enfield EN2",600000,4 bed terraced house for sale,Ideal Place,https://www.zoopla.co.uk/find-agents/branch/id...,https://www.zoopla.co.uk/for-sale/details/cont...,https://www.zoopla.co.uk/for-sale/details/6472...,4,2,2.0,terraced,sale,EN2


In [52]:
zoopla_sale.to_csv('zoopla_sale_master.csv',index = False)

In [53]:
zoopla_rent.drop(columns = ['Unnamed: 0'],inplace = True)

In [54]:
zoopla_rent.shape

(1323, 10)

In [55]:
zoopla_rent = zoopla_rent.dropna(subset = ['Price','No_of_Bedroom','No_of_Bathroom'],axis = 0)
zoopla_rent['Price_pcm'] = zoopla_rent['Price'].str.extract('([\d{1,}]+)')
zoopla_rent['Price_pw'] = zoopla_rent['Price'].str.extract('pcm£([\d{1,}]+)')
zoopla_rent['Price_pcm'] = zoopla_rent['Price_pcm'].str.replace(',','')
zoopla_rent['Price_pw'] = zoopla_rent['Price_pw'].str.replace(',','')
zoopla_rent['Price_pw'] = zoopla_rent['Price_pw'].astype('int')
zoopla_rent['Price_pcm'] = zoopla_rent['Price_pcm'].astype('int')
zoopla_rent['No_of_Bedroom'] = zoopla_rent['No_of_Bedroom'].astype('int')
zoopla_rent['No_of_Bathroom'] = zoopla_rent['No_of_Bathroom'].astype('int') 

In [56]:
zoopla_rent.dtypes

Address              object
Price                object
Description          object
Agent_name           object
Agent_website        object
Agent_contact        object
Listing_url          object
No_of_Bedroom         int32
No_of_Bathroom        int32
No_of_livingroom    float64
Price_pcm             int32
Price_pw              int32
dtype: object

In [57]:
zoopla_rent.drop(columns = ['Price'],inplace = True)

In [58]:
zoopla_rent

,Address,Description,Agent_name,Agent_website,Agent_contact,Listing_url,No_of_Bedroom,No_of_Bathroom,No_of_livingroom,Price_pcm,Price_pw
0,"Marsh Wall, London E14",1 bed flat to rent,AD and Partners,https://www.zoopla.co.uk/find-agents/branch/ad...,https://www.zoopla.co.uk/to-rent/details/conta...,https://www.zoopla.co.uk/to-rent/details/64733...,1,1,1.0,3683,850
3,"Colville Gardens, London W11",Studio to rent,Homelike Internet GmbH,https://www.zoopla.co.uk/find-agents/branch/ho...,https://www.zoopla.co.uk/to-rent/details/conta...,https://www.zoopla.co.uk/to-rent/details/57653...,1,1,NaN,2300,531
5,"Avis Square, London E1",4 bed terraced house to rent,Right Room,https://www.zoopla.co.uk/find-agents/branch/ri...,https://www.zoopla.co.uk/to-rent/details/conta...,https://www.zoopla.co.uk/to-rent/details/64732...,4,1,1.0,3600,831
6,"Hendon Way, Hendon NW4",2 bed maisonette to rent,Brampton Real Estate,https://www.zoopla.co.uk/find-agents/branch/br...,https://www.zoopla.co.uk/to-rent/details/conta...,https://www.zoopla.co.uk/to-rent/details/64732...,2,2,1.0,1950,450
8,"Ealing Road, Wembley, Middlesex HA0",2 bed flat to rent,Right Home Estate Agents,https://www.zoopla.co.uk/find-agents/branch/ri...,https://www.zoopla.co.uk/to-rent/details/conta...,https://www.zoopla.co.uk/to-rent/details/64732...,2,1,NaN,2100,485
...,...,...,...,...,...,...,...,...,...,...,...
1316,"St. Pauls Avenue, London NW2",1 bed flat to rent,OpenRent,https://www.zoopla.co.uk/find-agents/branch/op...,https://www.zoopla.co.uk/to-rent/details/conta...,https://www.zoopla.co.uk/to-rent/details/64725...,1,1,NaN,1650,381
1317,"Rotherhithe Street, London SE16",1 bed flat to rent,OpenRent,https://www.zoopla.co.uk/find-agents/branch/op...,https://www.zoopla.co.uk/to-rent/details/conta...,https://www.zoopla.co.uk/to-rent/details/64725...,1,1,NaN,999,231
1318,"Golders Green Road, London NW11",1 bed flat to rent,OpenRent,https://www.zoopla.co.uk/find-agents/branch/op...,https://www.zoopla.co.uk/to-rent/details/conta...,https://www.zoopla.co.uk/to-rent/details/64725...,1,2,NaN,825,190
1319,"St. Hildas Mount, London NW7",3 bed flat to rent,OpenRent,https://www.zoopla.co.uk/find-agents/branch/op...,https://www.zoopla.co.uk/to-rent/details/conta...,https://www.zoopla.co.uk/to-rent/details/64725...,3,2,NaN,2600,600


In [59]:
zoopla_rent['Property_type'] = zoopla_rent['Description'].str.extract('([a-zA-Z]+\s+[a-zA-Z.\-]+)',expand = True)

In [60]:
zoopla_rent['Property_type'] = zoopla_rent['Property_type'].str.replace('bed','')

In [61]:
zoopla_rent['Property_type'] = zoopla_rent['Property_type'].str.strip()

In [62]:
zoopla_rent['Property_type'] = zoopla_rent['Property_type'].apply(lambda x:x.split(' ')[0])

In [63]:
zoopla_rent['Transaction_type'] = np.repeat('rent',zoopla_rent.shape[0])

In [64]:
#checking for Duplicates
zoopla_rent.duplicated().sum()

22

In [65]:
#Resetting index
zoopla_rent.reset_index(drop = True,inplace = True)

In [66]:
# Extracting out the district Postcode present in the Address by using the list of District Postcode in London.
for i, j in enumerate(zoopla_rent['Address']):
    for k in post_list:
        if k in zoopla_rent['Address'][i]:
            zoopla_rent.loc[i, 'Postcode_district'] = k
            break
        else:
            zoopla_rent.loc[i, 'Postcode_district'] = np.NaN

# Filling null values with "None"
zoopla_rent['Postcode_district'].fillna('None', inplace=True)

# Making a copy of the District code yet to be gotten.
zoopla_rent_copy = zoopla_rent.query('Postcode_district == "None"')

# Dropping the index where we have district code to be null
zoopla_rent.drop(zoopla_rent_copy.index, inplace=True)

# Resetting index of the copied data
zoopla_rent_copy.reset_index(drop=True, inplace=True)

# Using geolocator to get other District codes
geolocator = Nominatim(user_agent="postcode_extractor")
for i, j in enumerate(zoopla_rent_copy['Address']):
    location = geolocator.geocode(zoopla_rent_copy['Address'][i], country_codes="GB", addressdetails=True,timeout = 10)
    if location is not None:
        postcode = location.raw.get('address', {}).get('postcode')
        if postcode:
            zoopla_rent_copy.loc[i, 'Postcode_district'] = postcode.split()[0]
        else:
            zoopla_rent_copy.loc[i, 'Postcode_district'] = 'None'
    else:
        zoopla_rent_copy.loc[i, 'Postcode_district'] = 'None'

# Checking district codes which are yet to be gotten
zoopla_rent_copy = zoopla_rent_copy.query('Postcode_district != "None"')

# Getting the final data
zoopla_rent = pd.concat([zoopla_rent, zoopla_rent_copy], ignore_index=True)

zoopla_rent


,Address,Description,Agent_name,Agent_website,Agent_contact,Listing_url,No_of_Bedroom,No_of_Bathroom,No_of_livingroom,Price_pcm,Price_pw,Property_type,Transaction_type,Postcode_district
0,"Marsh Wall, London E14",1 bed flat to rent,AD and Partners,https://www.zoopla.co.uk/find-agents/branch/ad...,https://www.zoopla.co.uk/to-rent/details/conta...,https://www.zoopla.co.uk/to-rent/details/64733...,1,1,1.0,3683,850,flat,rent,E1
1,"Colville Gardens, London W11",Studio to rent,Homelike Internet GmbH,https://www.zoopla.co.uk/find-agents/branch/ho...,https://www.zoopla.co.uk/to-rent/details/conta...,https://www.zoopla.co.uk/to-rent/details/57653...,1,1,NaN,2300,531,Studio,rent,W11
2,"Avis Square, London E1",4 bed terraced house to rent,Right Room,https://www.zoopla.co.uk/find-agents/branch/ri...,https://www.zoopla.co.uk/to-rent/details/conta...,https://www.zoopla.co.uk/to-rent/details/64732...,4,1,1.0,3600,831,terraced,rent,E1
3,"Hendon Way, Hendon NW4",2 bed maisonette to rent,Brampton Real Estate,https://www.zoopla.co.uk/find-agents/branch/br...,https://www.zoopla.co.uk/to-rent/details/conta...,https://www.zoopla.co.uk/to-rent/details/64732...,2,2,1.0,1950,450,maisonette,rent,NW4
4,"Ealing Road, Wembley, Middlesex HA0",2 bed flat to rent,Right Home Estate Agents,https://www.zoopla.co.uk/find-agents/branch/ri...,https://www.zoopla.co.uk/to-rent/details/conta...,https://www.zoopla.co.uk/to-rent/details/64732...,2,1,NaN,2100,485,flat,rent,HA0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,"Rotherhithe Street, London SE16",1 bed flat to rent,OpenRent,https://www.zoopla.co.uk/find-agents/branch/op...,https://www.zoopla.co.uk/to-rent/details/conta...,https://www.zoopla.co.uk/to-rent/details/64725...,1,1,NaN,999,231,flat,rent,E1
800,"Golders Green Road, London NW11",1 bed flat to rent,OpenRent,https://www.zoopla.co.uk/find-agents/branch/op...,https://www.zoopla.co.uk/to-rent/details/conta...,https://www.zoopla.co.uk/to-rent/details/64725...,1,2,NaN,825,190,flat,rent,NW1
801,"St. Hildas Mount, London NW7",3 bed flat to rent,OpenRent,https://www.zoopla.co.uk/find-agents/branch/op...,https://www.zoopla.co.uk/to-rent/details/conta...,https://www.zoopla.co.uk/to-rent/details/64725...,3,2,NaN,2600,600,flat,rent,NW7
802,"Embassy Gardens, London SW11",2 bed flat to rent,One Move Property Group Ltd,https://www.zoopla.co.uk/find-agents/branch/on...,https://www.zoopla.co.uk/to-rent/details/conta...,https://www.zoopla.co.uk/to-rent/details/62487...,2,2,1.0,3900,900,flat,rent,SW11


In [67]:
zoopla_rent.to_csv('zoopla_rent_master.csv',index = False)

<p id='wotm'></p>

## OTM

In [68]:
#onthemove data
otm_sale = pd.read_csv('otm_sale.csv')
otm_rent = pd.read_csv('otm_rent.csv')

In [69]:
otm_sale.drop(columns = ['Unnamed: 0'],inplace = True)

In [70]:
otm_sale.head()

,Price,Address,description,No_of_bedroom,No_of_bathroom,listing_url,agent_website,agent_name
0,"£4,750,000","Frognal Lane, Hampstead, NW3",4 bedroom house for sale,4,2,https://www.onthemarket.com/details/13184464/,https://www.onthemarket.com/agents/branch/tk-i...,TK International - Hampstead
1,"£800,000","Stephendale Road, Fulham, London, SW6",2 bedroom flat for sale,2,2,https://www.onthemarket.com/details/13262722/,https://www.onthemarket.com/agents/branch/aspi...,Aspire - Fulham South
2,"£400,000","Broadfield Road, Catford",2 bedroom flat for sale,2,1,https://www.onthemarket.com/details/13262059/,https://www.onthemarket.com/agents/branch/kinl...,Kinleigh Folkard & Hayward - Catford Sales
3,"£2,600,000","Millbank, London SW1P",2 bedroom flat for sale,2,2,https://www.onthemarket.com/details/13261993/,https://www.onthemarket.com/agents/branch/trev...,Trevlyn Properties - Maidstone
4,"£575,000","The Downs, Wimbledon",2 bedroom flat for sale,2,1,https://www.onthemarket.com/details/13261935/,https://www.onthemarket.com/agents/branch/kinl...,Kinleigh Folkard & Hayward - Wimbledon Sales


In [71]:
otm_sale.dtypes

Price             object
Address           object
description       object
No_of_bedroom     object
No_of_bathroom    object
listing_url       object
agent_website     object
agent_name        object
dtype: object

In [72]:
otm_sale = otm_sale.dropna(subset = ['Price','No_of_bedroom','No_of_bathroom'],axis = 0)
otm_sale['Price'] = otm_sale['Price'].str.extract('([\d{1,}]+)')
otm_sale['Price'] = otm_sale['Price'].str.replace(',','')
otm_sale['Price'] = otm_sale['Price'].astype('int')
otm_sale['No_of_bedroom'] = otm_sale['No_of_bedroom'].str.extract('([\d{1,2}]+)')
otm_sale['No_of_bathroom'] = otm_sale['No_of_bathroom'].str.extract('([\d{1,}]+)')
otm_sale = otm_sale.dropna(subset = ['Price','No_of_bedroom','No_of_bathroom'],axis = 0)
otm_sale['No_of_bedroom'] = otm_sale['No_of_bedroom'].astype('int')
otm_sale['No_of_bathroom'] = otm_sale['No_of_bathroom'].astype('int') 

In [73]:
otm_sale.reset_index(drop = True,inplace = True)

In [74]:
otm_sale.head()

,Price,Address,description,No_of_bedroom,No_of_bathroom,listing_url,agent_website,agent_name
0,4750000,"Frognal Lane, Hampstead, NW3",4 bedroom house for sale,4,2,https://www.onthemarket.com/details/13184464/,https://www.onthemarket.com/agents/branch/tk-i...,TK International - Hampstead
1,800000,"Stephendale Road, Fulham, London, SW6",2 bedroom flat for sale,2,2,https://www.onthemarket.com/details/13262722/,https://www.onthemarket.com/agents/branch/aspi...,Aspire - Fulham South
2,400000,"Broadfield Road, Catford",2 bedroom flat for sale,2,1,https://www.onthemarket.com/details/13262059/,https://www.onthemarket.com/agents/branch/kinl...,Kinleigh Folkard & Hayward - Catford Sales
3,2600000,"Millbank, London SW1P",2 bedroom flat for sale,2,2,https://www.onthemarket.com/details/13261993/,https://www.onthemarket.com/agents/branch/trev...,Trevlyn Properties - Maidstone
4,575000,"The Downs, Wimbledon",2 bedroom flat for sale,2,1,https://www.onthemarket.com/details/13261935/,https://www.onthemarket.com/agents/branch/kinl...,Kinleigh Folkard & Hayward - Wimbledon Sales


In [75]:
otm_sale.rename(columns = {'description':'Description'},inplace = True)

In [76]:
otm_sale['Property_type'] = otm_sale['Description'].str.extract('([a-zA-Z]+\s+[a-zA-Z.\-]+)',expand = True)

In [77]:
otm_sale['Property_type'] = otm_sale['Property_type'].str.replace('bedroom','')

In [78]:
otm_sale['Transaction_type'] = np.repeat('sale',otm_sale.shape[0])

In [79]:
otm_sale.rename(columns = {'No_of_bedroom':'No_of_Bedroom','No_of_bathroom':'No_of_Bathroom'},inplace = True)

In [80]:
#checking for Duplicates
otm_sale.duplicated().sum()

80

In [81]:
#Dropping Duplicates
otm_sale = otm_sale.drop_duplicates()

In [82]:
otm_sale.reset_index(drop = True,inplace = True)

In [83]:
otm_sale.head()

,Price,Address,Description,No_of_Bedroom,No_of_Bathroom,listing_url,agent_website,agent_name,Property_type,Transaction_type
0,4750000,"Frognal Lane, Hampstead, NW3",4 bedroom house for sale,4,2,https://www.onthemarket.com/details/13184464/,https://www.onthemarket.com/agents/branch/tk-i...,TK International - Hampstead,house,sale
1,800000,"Stephendale Road, Fulham, London, SW6",2 bedroom flat for sale,2,2,https://www.onthemarket.com/details/13262722/,https://www.onthemarket.com/agents/branch/aspi...,Aspire - Fulham South,flat,sale
2,400000,"Broadfield Road, Catford",2 bedroom flat for sale,2,1,https://www.onthemarket.com/details/13262059/,https://www.onthemarket.com/agents/branch/kinl...,Kinleigh Folkard & Hayward - Catford Sales,flat,sale
3,2600000,"Millbank, London SW1P",2 bedroom flat for sale,2,2,https://www.onthemarket.com/details/13261993/,https://www.onthemarket.com/agents/branch/trev...,Trevlyn Properties - Maidstone,flat,sale
4,575000,"The Downs, Wimbledon",2 bedroom flat for sale,2,1,https://www.onthemarket.com/details/13261935/,https://www.onthemarket.com/agents/branch/kinl...,Kinleigh Folkard & Hayward - Wimbledon Sales,flat,sale


In [84]:
# Extracting out the district Postcode present in the Address by using the list of District Postcode in London.
for i, j in enumerate(otm_sale['Address']):
    for k in post_list:
        if k in otm_sale['Address'][i]:
            otm_sale.loc[i, 'Postcode_district'] = k
            break
        else:
            otm_sale.loc[i, 'Postcode_district'] = np.NaN

# Filling null values with "None"
otm_sale['Postcode_district'].fillna('None', inplace=True)

# Making a copy of the District code yet to be gotten.
otm_sale_copy = otm_sale.query('Postcode_district == "None"')

# Dropping the index where we have district code to be null
otm_sale.drop(otm_sale_copy.index, inplace=True)

# Resetting index of the copied data
otm_sale_copy.reset_index(drop=True, inplace=True)

# Using geolocator to get other District codes
geolocator = Nominatim(user_agent="postcode_extractor")
for i, j in enumerate(otm_sale_copy['Address']):
    location = geolocator.geocode(otm_sale_copy['Address'][i], country_codes="GB", addressdetails=True,timeout = 10)
    if location is not None:
        postcode = location.raw.get('address', {}).get('postcode')
        if postcode:
            otm_sale_copy.loc[i, 'Postcode_district'] = postcode.split()[0]
        else:
            otm_sale_copy.loc[i, 'Postcode_district'] = 'None'
    else:
        otm_sale_copy.loc[i, 'Postcode_district'] = 'None'

# Checking district codes which are yet to be gotten
otm_sale_copy = otm_sale_copy.query('Postcode_district != "None"')

# Getting the final data
otm_sale = pd.concat([otm_sale, otm_sale_copy], ignore_index=True)

otm_sale


,Price,Address,Description,No_of_Bedroom,No_of_Bathroom,listing_url,agent_website,agent_name,Property_type,Transaction_type,Postcode_district
0,4750000,"Frognal Lane, Hampstead, NW3",4 bedroom house for sale,4,2,https://www.onthemarket.com/details/13184464/,https://www.onthemarket.com/agents/branch/tk-i...,TK International - Hampstead,house,sale,NW3
1,800000,"Stephendale Road, Fulham, London, SW6",2 bedroom flat for sale,2,2,https://www.onthemarket.com/details/13262722/,https://www.onthemarket.com/agents/branch/aspi...,Aspire - Fulham South,flat,sale,SW6
2,2600000,"Millbank, London SW1P",2 bedroom flat for sale,2,2,https://www.onthemarket.com/details/13261993/,https://www.onthemarket.com/agents/branch/trev...,Trevlyn Properties - Maidstone,flat,sale,SW1P
3,250000,"Cundy Road, London E16",1 bedroom flat for sale,1,1,https://www.onthemarket.com/details/13261889/,https://www.onthemarket.com/agents/branch/trev...,Trevlyn Properties - Maidstone,flat,sale,E1
4,7500000,"Gloucester Square, London, W2.",7 bedroom terraced house for sale,7,4,https://www.onthemarket.com/details/13261878/,https://www.onthemarket.com/agents/branch/knig...,Knight Frank - Hyde Park,terraced,sale,W2
...,...,...,...,...,...,...,...,...,...,...,...
674,400000,"Mare Street, Hackney",1 bedroom flat for sale,1,1,https://www.onthemarket.com/details/13256622/,https://www.onthemarket.com/agents/branch/wild...,Wild & Co. - Hackney,flat,sale,E8
675,650000,"Ingleby Gardens, Chigwell",3 bedroom semi-detached house for sale,3,1,https://www.onthemarket.com/details/13152350/,https://www.onthemarket.com/agents/branch/bowd...,Bowden Bradley - Essex,semi-detached,sale,IG7
676,350000,"Inglewood Close, Hainault",3 bedroom terraced house for sale,3,1,https://www.onthemarket.com/details/13114711/,https://www.onthemarket.com/agents/branch/bowd...,Bowden Bradley - Essex,terraced,sale,IG6
677,350000,"Turner Avenue, London",4 bedroom maisonette for sale,4,1,https://www.onthemarket.com/details/13256406/,https://www.onthemarket.com/agents/branch/bowd...,Bowden Bradley - Essex,maisonette,sale,N15


In [85]:
otm_sale.rename(columns = {'listing_url':'Listing_url','agent_website':'Agent_website',
                                'agent_name':'Agent_name'},inplace = True)

In [86]:
otm_sale.to_csv('otm_sale_master.csv',index = False)

In [87]:
otm_rent

,Unnamed: 0,Price,Address,description,No_of_bedroom,No_of_bathroom,listing_url,agent_website,agent_name
0,0,"£14,083 pcm (£3,250 pw)","Stafford Terrace, Kensington, London",3 bedroom flat to rent,3,2,https://www.onthemarket.com/details/11867187/,https://www.onthemarket.com/agents/branch/ches...,Chestertons - Kensington High Street
1,1,"£3,467 pcm (£800 pw)","Bathurst Mews, Tyburnia, London, W2",2 bedroom mews to rent,2,2,https://www.onthemarket.com/details/2601555/,https://www.onthemarket.com/agents/branch/luro...,Lurot Brand - Hyde Park
2,2,"£2,250 pcm (£519 pw)","Mandalay Road, London, SW4",2 bedroom flat to rent,2,1,https://www.onthemarket.com/details/13262721/,https://www.onthemarket.com/agents/branch/aspi...,Aspire - Clapham High Street
3,3,"£2,000 pcm (£462 pw)","Wellsmoor Gardens, Bickley",3 bedroom terraced house to rent,3,1,https://www.onthemarket.com/details/9422630/,https://www.onthemarket.com/agents/branch/hump...,Humphriss & Ryde - Chislehurst
4,4,"£1,350 pcm (£312 pw)","Saville Row, Hayes, Bromley",1 bedroom end of terrace house to rent,1,1,https://www.onthemarket.com/details/13262682/,https://www.onthemarket.com/agents/branch/hump...,Humphriss & Ryde - Chislehurst
...,...,...,...,...,...,...,...,...,...
961,961,"£1,995 pcm (£460 pw)","Nicoll Road, London",2 bedroom flat to rent,2,1,https://www.onthemarket.com/details/13258607/,https://www.onthemarket.com/agents/branch/suns...,Sunshine Estates - London
962,962,"£3,600 pcm (£831 pw)","Ashfield Road, London",4 bedroom house to rent,4,1,https://www.onthemarket.com/details/13258602/,https://www.onthemarket.com/agents/branch/suns...,Sunshine Estates - London
963,963,"£1,400 pcm (£323 pw)","Lancaster Road, Barnet",1 bedroom flat to rent,1,1,https://www.onthemarket.com/details/13258598/,https://www.onthemarket.com/agents/branch/suns...,Sunshine Estates - London
964,964,"£1,800 pcm (£415 pw)","Avenue Road, London",2 bedroom flat to rent,2,1,https://www.onthemarket.com/details/13258597/,https://www.onthemarket.com/agents/branch/suns...,Sunshine Estates - London


In [88]:
otm_rent.drop(columns = ['Unnamed: 0'],inplace = True)

In [89]:
otm_rent = otm_rent.dropna(subset = ['Price','No_of_bedroom','No_of_bathroom'],axis = 0)
otm_rent['Price_pcm'] = otm_rent['Price'].str.extract('([\d{1,}]+)')
otm_rent['Price_pw'] = otm_rent['Price'].str.extract('pcm \W+([\d{1,}]+)')
otm_rent['Price_pcm'] = otm_rent['Price_pcm'].str.replace(',','')
otm_rent['Price_pw'] = otm_rent['Price_pw'].str.replace(',','')
otm_rent['Price_pw'] = otm_rent['Price_pw'].astype('int')
otm_rent['Price_pcm'] = otm_rent['Price_pcm'].astype('int')
otm_rent['No_of_bedroom'] = otm_rent['No_of_bedroom'].str.extract('([\d{1,2}]+)')
otm_rent['No_of_bathroom'] = otm_rent['No_of_bathroom'].str.extract('([\d{1,}]+)')
otm_rent = otm_rent.dropna(subset = ['Price','No_of_bedroom','No_of_bathroom'],axis = 0)
otm_rent['No_of_bedroom'] = otm_rent['No_of_bedroom'].astype('int')
otm_rent['No_of_bathroom'] = otm_rent['No_of_bathroom'].astype('int') 

In [90]:
otm_rent.reset_index(drop = True,inplace = True)

In [91]:
otm_rent.rename(columns = {'No_of_bedroom':'No_of_Bedroom','No_of_bathroom':'No_of_Bathroom'},inplace = True)

In [92]:
otm_rent.rename(columns = {'description':'Description'},inplace = True)

In [93]:
otm_rent['Property_type'] = otm_rent['Description'].str.extract('([a-zA-Z]+\s+[a-zA-Z.\-]+)',expand = True)

In [94]:
otm_rent['Property_type'] = otm_rent['Property_type'].str.replace('bedroom','')

In [95]:
otm_rent['Transaction_type'] = np.repeat('rent',otm_rent.shape[0])

In [96]:
otm_rent.drop('Price',axis = 1,inplace = True)

In [97]:
#checking for duplicates
otm_rent.duplicated().sum()

198

In [98]:
#Dropping duplicates
otm_rent = otm_rent.drop_duplicates()

In [99]:
otm_rent.reset_index(drop = True,inplace = True)

In [100]:
# Extracting out the district Postcode present in the Address by using the list of District Postcode in London.
for i, j in enumerate(otm_rent['Address']):
    for k in post_list:
        if k in otm_rent['Address'][i]:
            otm_rent.loc[i, 'Postcode_district'] = k
            break
        else:
            otm_rent.loc[i, 'Postcode_district'] = np.NaN

# Filling null values with "None"
otm_rent['Postcode_district'].fillna('None', inplace=True)

# Making a copy of the District code yet to be gotten.
otm_rent_copy = otm_rent.query('Postcode_district == "None"')

# Dropping the index where we have district code to be null
otm_rent.drop(otm_rent_copy.index, inplace=True)

# Resetting index of the copied data
otm_rent_copy.reset_index(drop=True, inplace=True)

# Using geolocator to get other District codes
geolocator = Nominatim(user_agent="postcode_extractor")
for i, j in enumerate(otm_rent_copy['Address']):
    location = geolocator.geocode(otm_rent_copy['Address'][i], country_codes="GB", addressdetails=True,timeout = 10)
    if location is not None:
        postcode = location.raw.get('address', {}).get('postcode')
        if postcode:
            otm_rent_copy.loc[i, 'Postcode_district'] = postcode.split()[0]
        else:
            otm_rent_copy.loc[i, 'Postcode_district'] = 'None'
    else:
        otm_rent_copy.loc[i, 'Postcode_district'] = 'None'

# Checking district codes which are yet to be gotten
otm_rent_copy = otm_rent_copy.query('Postcode_district != "None"')

# Getting the final data
otm_rent = pd.concat([otm_rent, otm_rent_copy], ignore_index=True)

otm_rent


,Address,Description,No_of_Bedroom,No_of_Bathroom,listing_url,agent_website,agent_name,Price_pcm,Price_pw,Property_type,Transaction_type,Postcode_district
0,"Bathurst Mews, Tyburnia, London, W2",2 bedroom mews to rent,2,2,https://www.onthemarket.com/details/2601555/,https://www.onthemarket.com/agents/branch/luro...,Lurot Brand - Hyde Park,3467,800,mews,rent,W2
1,"Mandalay Road, London, SW4",2 bedroom flat to rent,2,1,https://www.onthemarket.com/details/13262721/,https://www.onthemarket.com/agents/branch/aspi...,Aspire - Clapham High Street,2250,519,flat,rent,SW4
2,"Warwick Place, London W5",2 bedroom cottage to rent,2,1,https://www.onthemarket.com/details/13261879/,https://www.onthemarket.com/agents/branch/tayl...,Taylor Smith - Acton,2100,485,cottage,rent,W5
3,"Staplehurst Court, London SE2",1 bedroom in a flat share to rent,1,1,https://www.onthemarket.com/details/13261578/,https://www.onthemarket.com/agents/branch/home...,Home World Management - London,750,173,in,rent,E2
4,"Sherbrooke Road, London, SW6",3 bedroom house to rent,3,2,https://www.onthemarket.com/details/11825147/,https://www.onthemarket.com/agents/branch/aspi...,Aspire - Fulham South,3250,750,house,rent,SW6
...,...,...,...,...,...,...,...,...,...,...,...,...
627,"Nicoll Road, London",2 bedroom flat to rent,2,1,https://www.onthemarket.com/details/13258607/,https://www.onthemarket.com/agents/branch/suns...,Sunshine Estates - London,1995,460,flat,rent,NW10
628,"Ashfield Road, London",4 bedroom house to rent,4,1,https://www.onthemarket.com/details/13258602/,https://www.onthemarket.com/agents/branch/suns...,Sunshine Estates - London,3600,831,house,rent,N4
629,"Lancaster Road, Barnet",1 bedroom flat to rent,1,1,https://www.onthemarket.com/details/13258598/,https://www.onthemarket.com/agents/branch/suns...,Sunshine Estates - London,1400,323,flat,rent,EN4
630,"Avenue Road, London",2 bedroom flat to rent,2,1,https://www.onthemarket.com/details/13258597/,https://www.onthemarket.com/agents/branch/suns...,Sunshine Estates - London,1800,415,flat,rent,BR3


In [101]:
otm_rent.rename(columns = {'listing_url':'Listing_url','agent_website':'Agent_website',
                                'agent_name':'Agent_name'},inplace = True)

In [102]:
otm_rent.to_csv('otm_rent_master.csv',index = False)

[GoToTop](#top)

<p id='model'></p>

## DATA MODELLING

In [103]:
#Creating a master list for all properties available for sale in London
sales = pd.concat([rightmove_sale,zoopla_sale,otm_sale])

In [104]:
sales

,Address,Property_type,Price,No_of_Bedroom,No_of_Bathroom,Description,Agent_name,Agent_website,Agent_contact,Listing_url,Transaction_type,Postcode_district,No_of_livingroom
0,"Glyn Road, London, E5",Terraced,1100000,3,1,A truly unique and stunning three bedroom peri...,"Stirling Ackroyd, Clapton",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1352692...,sales,E5,NaN
1,"Lygon Place, Belgravia, SW1W",House,45000000,7,9,Ref. LOB0798 - Set behind a gated Belgravia dr...,"Beauchamp Estates Ltd, Mayfair - Resale",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1293029...,sales,SW1W,NaN
2,"Mayfair Freehold House, Park Lane Area, W1K",House,45000000,12,18,This stunning 12 bedroom (including 3 staff be...,"Luxury Living Homes International, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1303065...,sales,W1K,NaN
3,"Pitt Street, Kensington, W8",House,44000000,6,5,Ref. LOB0625 - This striking and unique house ...,"Beauchamp Estates Ltd, Mayfair - Resale",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1318567...,sales,W8,NaN
4,"Pitt Street, London, W8",Detached,44000000,7,5,"A rare opportunity to own this unique, contemp...","Savills, Kensington",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1318464...,sales,W8,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
674,"Mare Street, Hackney",flat,400000,1,1,1 bedroom flat for sale,Wild & Co. - Hackney,https://www.onthemarket.com/agents/branch/wild...,NaN,https://www.onthemarket.com/details/13256622/,sale,E8,NaN
675,"Ingleby Gardens, Chigwell",semi-detached,650000,3,1,3 bedroom semi-detached house for sale,Bowden Bradley - Essex,https://www.onthemarket.com/agents/branch/bowd...,NaN,https://www.onthemarket.com/details/13152350/,sale,IG7,NaN
676,"Inglewood Close, Hainault",terraced,350000,3,1,3 bedroom terraced house for sale,Bowden Bradley - Essex,https://www.onthemarket.com/agents/branch/bowd...,NaN,https://www.onthemarket.com/details/13114711/,sale,IG6,NaN
677,"Turner Avenue, London",maisonette,350000,4,1,4 bedroom maisonette for sale,Bowden Bradley - Essex,https://www.onthemarket.com/agents/branch/bowd...,NaN,https://www.onthemarket.com/details/13256406/,sale,N15,NaN


In [105]:
#Merging the List with the Postcode data available so as to get the exact location of these Properties
sales_data =  pd.merge(postcode_dist,sales,left_on = 'Postcode district',right_on = 'Postcode_district',how = 'inner')

In [106]:
#Checking for duplicated properties.
sales_data.duplicated().sum()

22

In [107]:
#Some website may be listed on Multiple websites, so we need to get a unique list of properties.
sales_data = sales_data.drop_duplicates()

In [108]:
sales_data

,Postcode district,Local Areas,Borough,Address,Property_type,Price,No_of_Bedroom,No_of_Bathroom,Description,Agent_name,Agent_website,Agent_contact,Listing_url,Transaction_type,Postcode_district,No_of_livingroom
0,BR1,"Bromley, Bickley, Downham",Bromley,"Spencer Road, Bromley BR1",town,525000,3,2,3 bed town house for sale,Proctors - Bromley,https://www.zoopla.co.uk/find-agents/branch/pr...,https://www.zoopla.co.uk/for-sale/details/cont...,https://www.zoopla.co.uk/for-sale/details/6472...,sale,BR1,2.0
1,BR1,"Bromley, Bickley, Downham",Bromley,"Rodway Road, Bromley BR1",flat,475000,2,2,2 bed flat for sale,Langford Russell - Bromley,https://www.zoopla.co.uk/find-agents/branch/la...,https://www.zoopla.co.uk/for-sale/details/cont...,https://www.zoopla.co.uk/for-sale/details/6472...,sale,BR1,1.0
2,BR1,"Bromley, Bickley, Downham",Bromley,"Rangefield Road, Downham, Bromley BR1",end,450000,3,1,3 bed end terrace house for sale,BR Estate Agent,https://www.zoopla.co.uk/find-agents/branch/br...,https://www.zoopla.co.uk/for-sale/details/cont...,https://www.zoopla.co.uk/for-sale/details/6472...,sale,BR1,2.0
3,BR1,"Bromley, Bickley, Downham",Bromley,"College Road, Bromley, BR1",detached,1150000,6,3,6 bedroom detached house for sale,Foxtons - Bromley,https://www.onthemarket.com/agents/branch/foxt...,NaN,https://www.onthemarket.com/details/13187264/,sale,BR1,NaN
4,BR1,"Bromley, Bickley, Downham",Bromley,"Faro Close, Bromley, BR1 2RR",flat,300000,1,1,1 bedroom flat for sale,Hunters - Chislehurst & Bromley,https://www.onthemarket.com/agents/branch/hunt...,NaN,https://www.onthemarket.com/details/13257555/,sale,BR1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2233,WC2N,Charing Cross,Westminster,"Strand, London",Apartment,6000000,3,3,3bedrooms 3bathrooms.,"CHARLES WILLIAM PROPERTY INVESTMENT, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1325462...,sales,WC2N,NaN
2234,WC2R,Somerset House,Westminster,"190, The Strand, WC2R",Apartment,6500000,3,3,A spacious and bright three bedroom apartment ...,"Citian & Partners, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1326518...,sales,WC2R,NaN
2235,WC2R,Somerset House,Westminster,"Strand, London, WC2R",Apartment,5999995,3,3,A spacious and bright three bedroom apartment ...,"Life Residential, County Hall - South Bank Sales",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1252798...,sales,WC2R,NaN
2236,WC2R,Somerset House,Westminster,"Strand, London, WC2R",Apartment,5999995,3,3,A spacious and bright three bedroom apartment ...,"Life Residential, County Hall - South Bank Sales",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-for-sale/...,https://www.rightmove.co.uk/properties/1252798...,sales,WC2R,NaN


In [109]:
sales_data.to_csv('master_sales.csv',index = False)

In [110]:
#Getting the master list of all properties available for rent.
rent = pd.concat([rightmove_rent,zoopla_rent,otm_rent])

In [111]:
rent

,Address,Property_type,Price_pcm,Price_pw,No_of_Bedroom,No_of_Bathroom,Description,Agent_name,Agent_website,Agent_contact,Listing_url,Postcode_district,Transaction_type,No_of_livingroom
0,"Blythwood Road, Stroud Green, London, N4",End of Terrace,5250,1212,5,2,NO DEPOSIT OPTION AVAILABLE. Lovely five bedro...,"Martyn Gerrard, Crouch End",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1352645...,N4,rent,NaN
1,"Kingfisher Heights, London, E16",Flat,1850,427,1,1,Property Description: This beautiful one bedro...,"OpenRent, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1354085...,E1,rent,NaN
2,"Torriano Avenue, London, NW5",Flat,4500,1038,1,1,A wonderfully stylish 1 bedroom apartment loca...,"OpenRent, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1354085...,NW5,rent,NaN
3,"Ringstead Road, London, SE6",Flat,1460,337,1,1,We are proud to offer this delightful 1 bedroo...,"OpenRent, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1354085...,E6,rent,NaN
4,"A Millway, London, NW7",Maisonette,1800,415,2,1,We are proud to offer this delightful 2 bedroo...,"OpenRent, London",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1354085...,NW7,rent,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627,"Nicoll Road, London",flat,1995,460,2,1,2 bedroom flat to rent,Sunshine Estates - London,https://www.onthemarket.com/agents/branch/suns...,NaN,https://www.onthemarket.com/details/13258607/,NW10,rent,NaN
628,"Ashfield Road, London",house,3600,831,4,1,4 bedroom house to rent,Sunshine Estates - London,https://www.onthemarket.com/agents/branch/suns...,NaN,https://www.onthemarket.com/details/13258602/,N4,rent,NaN
629,"Lancaster Road, Barnet",flat,1400,323,1,1,1 bedroom flat to rent,Sunshine Estates - London,https://www.onthemarket.com/agents/branch/suns...,NaN,https://www.onthemarket.com/details/13258598/,EN4,rent,NaN
630,"Avenue Road, London",flat,1800,415,2,1,2 bedroom flat to rent,Sunshine Estates - London,https://www.onthemarket.com/agents/branch/suns...,NaN,https://www.onthemarket.com/details/13258597/,BR3,rent,NaN


In [112]:
#Getting the exact location of where they are located by merging with the District zipcode of London available.
rent_data =  pd.merge(postcode_dist,rent,left_on = 'Postcode district',right_on = 'Postcode_district',how = 'inner')

In [113]:
#Checking for multiple listing.
rent_data.duplicated().sum()

126

In [114]:
#Removing multiple listing
rent_data = rent_data.drop_duplicates()

In [115]:
rent_data

,Postcode district,Local Areas,Borough,Address,Property_type,Price_pcm,Price_pw,No_of_Bedroom,No_of_Bathroom,Description,Agent_name,Agent_website,Agent_contact,Listing_url,Postcode_district,Transaction_type,No_of_livingroom
0,BR1,"Bromley, Bickley, Downham",Bromley,"Chelford Road, Bromley BR1",flat,1700,392,2,2,2 bed flat to rent,Andrew Reeves Countrywide - Bromley,https://www.zoopla.co.uk/find-agents/branch/an...,https://www.zoopla.co.uk/to-rent/details/conta...,https://www.zoopla.co.uk/to-rent/details/64732...,BR1,rent,1.0
2,BR1,"Bromley, Bickley, Downham",Bromley,"Widmore Road, Bromley BR1",flat,1900,438,2,2,2 bed flat to rent,Leo Newman,https://www.zoopla.co.uk/find-agents/branch/le...,https://www.zoopla.co.uk/to-rent/details/conta...,https://www.zoopla.co.uk/to-rent/details/64728...,BR1,rent,1.0
3,BR1,"Bromley, Bickley, Downham",Bromley,"Palace Road, Bromley BR1",property,1600,369,2,1,2 bed property to rent,Andrew Reeves Countrywide - Bromley,https://www.zoopla.co.uk/find-agents/branch/an...,https://www.zoopla.co.uk/to-rent/details/conta...,https://www.zoopla.co.uk/to-rent/details/55479...,BR1,rent,1.0
4,BR1,"Bromley, Bickley, Downham",Bromley,"Widmore Road, Bromley, BR1",apartment,1900,438,2,2,2 bedroom apartment to rent,Leo Newman - London,https://www.onthemarket.com/agents/branch/leo-...,NaN,https://www.onthemarket.com/details/13262988/,BR1,rent,NaN
5,BR1,"Bromley, Bickley, Downham",Bromley,"Widmore Road, Bromley, BR1",apartment,1550,358,1,1,1 bedroom apartment to rent,Leo Newman - London,https://www.onthemarket.com/agents/branch/leo-...,NaN,https://www.onthemarket.com/details/13259222/,BR1,rent,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2365,WC2N,Charing Cross,Westminster,"Craven Street, Charing Cross WC2N",flat,2037,470,1,1,1 bed flat to rent,SN Estates London,https://www.zoopla.co.uk/find-agents/branch/sn...,https://www.zoopla.co.uk/to-rent/details/conta...,https://www.zoopla.co.uk/to-rent/details/43648...,WC2N,rent,NaN
2366,WC2R,Somerset House,Westminster,"Strand, London, WC2R",Apartment,7150,1650,2,2,Beautifully presented two bedroom apartment wi...,"Savills Lettings, Mayfair",https://www.rightmove.co.uk/estate-agents/agen...,https://www.rightmove.co.uk/property-to-rent/c...,https://www.rightmove.co.uk/properties/1353678...,WC2R,rent,NaN
2367,WC2R,Somerset House,Westminster,"Strand, The Strand, London WC2R",flat,6500,1500,1,1,1 bed flat to rent,Foxtons - West End,https://www.zoopla.co.uk/find-agents/branch/fo...,https://www.zoopla.co.uk/to-rent/details/conta...,https://www.zoopla.co.uk/to-rent/details/64731...,WC2R,rent,1.0
2368,WC2R,Somerset House,Westminster,"Gladstone House, 190 The Strand, 190 The Stran...",flat,7150,1650,2,2,2 bed flat to rent,Chase Apartments,https://www.zoopla.co.uk/find-agents/branch/ch...,https://www.zoopla.co.uk/to-rent/details/conta...,https://www.zoopla.co.uk/to-rent/details/64731...,WC2R,rent,1.0


In [116]:
rent_data.to_csv('master_rent.csv',index = False)

## END NOTE

I'm delighted to have completed this project, and it has been more than just a competition for me. Throughout this project, I had the opportunity to enhance my analytical and programming skills. Among the three websites, I encountered the most challenges when gathering data from Zoopla due to its bot detection measures. However, with extensive research and learning, I managed to overcome these obstacles and find a way forward. This experience has exposed me to a variety of web scraping packages, expanding my knowledge in this area. I extend my heartfelt appreciation to the organizers of this competition for granting me the opportunity to participate.